In [4]:
#Bibliotecas e Variaveis Uteis
import geopandas as gpd
from calendar import monthrange
from collections import defaultdict
import shutil
import matplotlib.pyplot as plt
from glob import glob
import gc
import pandas as pd
import numpy as np
import xarray as xr
import pyarrow.parquet as pa_parquet
import cdsapi, backoff, zipfile, shutil, time
import os, json
import time
from glob import glob
from tqdm import tqdm
import netCDF4
import h5netcdf
import requests
import fastparquet
import dask.dataframe as dd
from pathlib import Path
import polars as pl
import hvplot.pandas
from polars import selectors as cs
import seaborn as sns

pasta_dados = r'C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados'

# 2. Lista de todas as pastas que precisam existir
# Extraímos apenas o diretório de cada caminho de arquivo definido
pastas_necessarias = [
    pasta_dados,
    os.path.join(pasta_dados, 'Series_Temporais'),
    os.path.join(pasta_dados, 'ERA5_Bruto'),
    os.path.join(pasta_dados, 'EDA'),
    os.path.join(pasta_dados, 'EDA', 'visualizacoes'),
    os.path.join(pasta_dados, 'EDA', 'consolidados')
]

# 3. Loop para criar as pastas automaticamente
print("Verificando e criando diretórios...")
for pasta in pastas_necessarias:
    os.makedirs(pasta, exist_ok=True)
    print(f"OK: {pasta}")

#Caminhos ruins
caminho_series_temporais = os.path.join(pasta_dados, 'Series_Temporais', 'series_temporais_bruto.parquet')
caminho_dados_estaticos = os.path.join(pasta_dados, 'Dataset_Locais.csv')
caminho_dataset_final = os.path.join(pasta_dados, 'Dataset_Final_Para_Modelagem.parquet')
caminho_dataset_intermediario = os.path.join(pasta_dados, 'Dataset_Intermediario.parquet')
output_path = os.path.join(pasta_dados, 'Series_Temporais')

#Limpo
pasta_era5 = os.path.join(pasta_dados, 'ERA5_Bruto')
serie_temporal = os.path.join(pasta_dados, 'Series_Temporais')
tratado_era5 = os.path.join(serie_temporal,  'series_temporais_bruto.parquet')
dataset_locais = os.path.join(pasta_dados, 'Dataset_Locais.csv')
#dataset_locais = os.path.join(pasta_dados, 'dataset_locais_semNaN.csv')

pq = pasta_dados + "//Series_Temporais//series_temporais_*.parquet"
PARQUETS = glob(pq)

#Limpeza e Trat dos dados
EDA_DIR = os.path.join(pasta_dados, 'EDA') #"/content/drive/MyDrive/PUC/IC/ColetaEra5/Dados/EDA"
os.makedirs(EDA_DIR, exist_ok=True)
viz_dir = os.path.join(EDA_DIR, "visualizacoes")
os.makedirs(viz_dir, exist_ok=True)

percent_glob = os.path.join(EDA_DIR, "percentis_globais.csv")
consolidado = os.path.join(EDA_DIR, "consolidados")
global_stat = os.path.join(consolidado,"estatisticas_globais.csv")
hist = os.path.join(consolidado, "histogramas_globais.csv")
valid = os.path.join(consolidado, "validacoes_detalhadas.csv")





#Variaveis Uteis

# Configuração de download
anos = range(2015, 2025); meses = [str(m).zfill(2) for m in range(1, 13)]
#anos = [2024]; meses = ['06']

subsistema_nordeste_ufs = [
    'MA',  # Maranhão
    'PI',  # Piauí
    'CE',  # Ceará
    'RN',  # Rio Grande do Norte
    'PB',  # Paraíba
    'PE',  # Pernambuco
    'AL',  # Alagoas
    'SE',  # Sergipe
    'BA'   # Bahia
]

status_operacional = [
    'Operação', 'Construção não iniciada', 'Construção', 'DRO'
    ]
#DRO -> Despacho de Requerimento de Outorga -> Entrara no longo prazo (talvez mais de 5 anos)
#Construção não iniciada -> Entrara no médio prazo (menos de 5 anos)
#Construção -> Entrara no curto prazo (de 1-3 anos)


Verificando e criando diretórios...
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\Series_Temporais
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\ERA5_Bruto
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\visualizacoes
OK: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\consolidados


# Coleta Parte 2 - ERA5

In [9]:
#Criação dos Datasets grandes
'''
# --- Definição dos Caminhos ---
pasta_dados = os.path.join('Dados', 'LocParquesEol1')

arquivos_para_processar = {
    "CentraisGeradoras": os.path.join(pasta_dados, 'Central_Geradora_Eólica_-_EOL.shp'),
    "Aerogeradores": os.path.join(pasta_dados, 'Aerogeradores.shp'),
    "AerogeradoresAlteracao": os.path.join(pasta_dados, 'Aerogeradores_com_solicitação_de_alteração_de_projetos.shp'),
    "PoligonosParques": os.path.join(pasta_dados, 'Polígono_do_Parque_Eólico_-_EOL.shp')
}

print("Iniciando extração de dados...")

# --- Loop de Processamento ---
for nome_saida, caminho_shapefile in arquivos_para_processar.items():

    print(f"\n--- Processando arquivo para: {nome_saida} ---")

    try:
        gdf = gpd.read_file(caminho_shapefile, encoding='utf-8')
        print(gdf.columns)

        df_final = None

        if nome_saida == "CentraisGeradoras":
            colunas_principais = ['CEG', 'Nome', 'P_FISC_KW', 'FASE', 'UF1', 'QTD_AEG', 'INICIO_OPE', 'geometry']
            df_final = gdf[colunas_principais].copy()

            # Adicionando e renomeando colunas para padronização
            df_final['potencia_mw'] = df_final['P_FISC_KW'] / 1000
            df_final.rename(columns={'Nome': 'nome_eol', 'FASE': 'status_operacao', 'UF1': 'uf', 'QTD_AEG':'qtd_aerogeradores'}, inplace=True)

            # Converte a data de início de operação para extrair apenas o ano
            df_final['ano_inicio_operacao'] = pd.to_datetime(df_final['INICIO_OPE'], errors='coerce').dt.year

            df_final['longitude'] = df_final.geometry.x
            df_final['latitude'] = df_final.geometry.y
            df_final.drop(columns=['geometry', 'P_FISC_KW', 'INICIO_OPE'], inplace=True)

            # Filtros
            df_final = df_final[df_final['uf'].isin(subsistema_nordeste_ufs)]
            df_final = df_final[df_final['status_operacao'].isin(status_operacional)]

        elif nome_saida == "Aerogeradores" or nome_saida == "AerogeradoresAlteracao":
            colunas_principais = ['CEG', 'ALT_TORRE', 'DIAM_ROTOR', 'POT_MW', 'NOME_EOL', 'DEN_AEG', 'OPERACAO', 'UF', 'geometry']
            df_final = gdf[colunas_principais].copy()

            df_final['longitude'] = df_final.geometry.x
            df_final['latitude'] = df_final.geometry.y

            df_final.rename(columns={
                'POT_MW': 'potencia_turbina_mw',
                'ALT_TORRE':'altura_torre_m',
                'DIAM_ROTOR':'diametro_rotor_m',
                'NOME_EOL': 'nome_eol',
                'DEN_AEG': 'denominacao_aerogerador',
                'OPERACAO': 'status_operacao_turbina',
                'UF': 'uf'
            }, inplace=True)
            df_final.drop(columns=['geometry'], inplace=True)

        elif nome_saida == "PoligonosParques":
            # Removida a coluna 'CEG' que não existe neste arquivo
            colunas_principais = ['NOME_EOL', 'POT_MW', 'AREA_M2', 'geometry']
            df_final = gdf[colunas_principais].copy()

            df_final['centroid_lon'] = df_final.geometry.centroid.x
            df_final['centroid_lat'] = df_final.geometry.centroid.y
            df_final.rename(columns={'NOME_EOL': 'nome_eol', 'POT_MW': 'potencia_mw', 'AREA_M2':'area_m2'}, inplace=True)


        if df_final is not None:
            caminho_saida = os.path.join(pasta_dados, f'{nome_saida}.csv')
            df_final.to_csv(caminho_saida, index=False, encoding='utf-8-sig')
            print(f"Dados extraídos com sucesso! {len(df_final)} registros salvos em '{caminho_saida}'")

    except Exception as e:
        print(f"!!!!!! Ocorreu um erro ao processar o arquivo para '{nome_saida}': {e}")
        continue

print("\n--- Extração de todos os arquivos concluída! ---")
'''

'\n# --- Definição dos Caminhos ---\npasta_dados = os.path.join(\'Dados\', \'LocParquesEol1\')\n\narquivos_para_processar = {\n    "CentraisGeradoras": os.path.join(pasta_dados, \'Central_Geradora_Eólica_-_EOL.shp\'),\n    "Aerogeradores": os.path.join(pasta_dados, \'Aerogeradores.shp\'),\n    "AerogeradoresAlteracao": os.path.join(pasta_dados, \'Aerogeradores_com_solicitação_de_alteração_de_projetos.shp\'),\n    "PoligonosParques": os.path.join(pasta_dados, \'Polígono_do_Parque_Eólico_-_EOL.shp\')\n}\n\nprint("Iniciando extração de dados...")\n\n# --- Loop de Processamento ---\nfor nome_saida, caminho_shapefile in arquivos_para_processar.items():\n\n    print(f"\n--- Processando arquivo para: {nome_saida} ---")\n\n    try:\n        gdf = gpd.read_file(caminho_shapefile, encoding=\'utf-8\')\n        print(gdf.columns)\n\n        df_final = None\n\n        if nome_saida == "CentraisGeradoras":\n            colunas_principais = [\'CEG\', \'Nome\', \'P_FISC_KW\', \'FASE\', \'UF1\', \'QTD_

In [10]:
#Tratando dataset_locais NaNs

turbinas = pd.read_csv(dataset_locais)

# --- PASSO 1: REMOVER DUPLICATAS ---

colunas_chave = ['CEG', 'denominacao_aerogerador', 'longitude_turbina', 'latitude_turbina']
linhas_antes = turbinas.shape[0]
turbinas = turbinas.drop_duplicates(subset=colunas_chave, keep='first')
print(f"{linhas_antes - turbinas.shape[0]} linhas duplicadas foram removidas.")


# --- PASSO 2: CORRIGIR OS NULOS EM TURBINAS EM 'OPERAÇÃO' ---
print("\nIniciando preenchimento para turbinas em 'Operação' com ano nulo...")

# 2.1. Tentar preencher com a mediana do parque (abordagem principal)
medianas_por_parque = turbinas.groupby('nome_parque')['ano_inicio_operacao'].transform('median')
turbinas['ano_inicio_operacao']= turbinas['ano_inicio_operacao'].fillna(medianas_por_parque)

# 2.2. Fallback: Para parques que continuam nulos (pois não tinham nenhuma data),
# usar a mediana GLOBAL de todas as turbinas em operação.
mediana_global_operacao = turbinas[turbinas['status_operacao'] == 'Operação']['ano_inicio_operacao'].median()
print(f"Mediana global para turbinas em operação é: {mediana_global_operacao:.0f}")
turbinas['ano_inicio_operacao']= turbinas['ano_inicio_operacao'].fillna(mediana_global_operacao)


# --- PASSO 3: PREENCHER OS DEMAIS STATUS (Construção, DRO, etc.) ---
print("\nIniciando preenchimento para turbinas em construção ou planejadas...")
ANO_ATUAL = 2025
prazos_em_anos = {
    'Construção': (1, 3),
    'Construção não iniciada': (3, 5),
    'DRO': (4, 8)
}

for status, (min_anos, max_anos) in prazos_em_anos.items():
    mascara = (turbinas['status_operacao'] == status) & (turbinas['ano_inicio_operacao'].isnull())
    num_a_preencher = mascara.sum()

    if num_a_preencher > 0:
        print(f"Preenchendo {num_a_preencher} turbinas com status '{status}'.")
        anos_adicionais = np.random.uniform(low=min_anos, high=max_anos, size=num_a_preencher)
        ano_estimado = ANO_ATUAL + anos_adicionais
        turbinas.loc[mascara, 'ano_inicio_operacao'] = ano_estimado


# --- PASSO 4: VERIFICAÇÃO FINAL ---
turbinas['ano_inicio_operacao'] = turbinas['ano_inicio_operacao'].round(1)

print("\n" + "="*50)
print("VERIFICAÇÃO FINAL")
print("="*50)

# Verificando se ainda existem valores nulos na coluna
nulos_restantes = turbinas['ano_inicio_operacao'].isnull().sum()
print(f"Número de valores nulos restantes em 'ano_inicio_operacao': {nulos_restantes}")

if nulos_restantes == 0:
    print("Sucesso! Todas as lacunas foram preenchidas.")
else:
    print("Atenção! Ainda existem lacunas não preenchidas.")

# Imprimindo o final do DataFrame, onde o problema era visível
print("\n--- Final do DataFrame (após todas as correções) ---")
print(turbinas[['nome_parque', 'status_operacao', 'ano_inicio_operacao']].tail(10))
saida= pasta_dados + '//dataset_locais_semNaN.csv'
turbinas.to_csv(saida, index=False, encoding='utf-8-sig')
dataset_locais= saida

13 linhas duplicadas foram removidas.

Iniciando preenchimento para turbinas em 'Operação' com ano nulo...
Mediana global para turbinas em operação é: 2017

Iniciando preenchimento para turbinas em construção ou planejadas...

VERIFICAÇÃO FINAL
Número de valores nulos restantes em 'ano_inicio_operacao': 0
Sucesso! Todas as lacunas foram preenchidas.

--- Final do DataFrame (após todas as correções) ---
                   nome_parque status_operacao  ano_inicio_operacao
17748  Ventos de São Rafael 03        Operação               2017.0
17749  Ventos de São Rafael 03        Operação               2017.0
17750  Ventos de São Rafael 03        Operação               2017.0
17751  Ventos de São Rafael 03        Operação               2017.0
17752  Ventos de São Rafael 03        Operação               2017.0
17753  Ventos de São Rafael 03        Operação               2017.0
17754  Ventos de São Rafael 03        Operação               2017.0
17755  Ventos de São Rafael 03        Operação    

In [5]:
# Coleta ERA5
#pasta_dados = os.path.join('Dados', 'LocParquesEol1')

try:
    import netCDF4
    print("netCDF4 disponível")
except ImportError:
    try:
        import h5netcdf
        print("h5netcdf disponível")
    except ImportError:
        print("AVISO: Nem netCDF4 nem h5netcdf estão instalados. Instale um deles:")
        print("pip install netcdf4")
        print("ou")
        print("pip install h5netcdf")



# Local de Salvamento

os.makedirs(pasta_era5, exist_ok=True)

df_locais = pd.read_csv(dataset_locais) if isinstance (dataset_locais, str) else dataset_locais.copy()

df_locais["lat_base"] = df_locais["latitude_turbina"].fillna(df_locais["latitude_parque"])
df_locais["lon_base"] = df_locais["longitude_turbina"].fillna(df_locais["longitude_parque"])

RES = 0.25

def snap_to_era5_grid(lat, lon, res=RES):
    lat_s = float(np.round(np.round(lat / res) * res, 2))
    lon_s = float(np.round(np.round(lon / res) * res, 2))
    return lat_s, lon_s



def bbox_single_cell(lat_era5: float, lon_era5: float, res: float = RES):
    half = res/2
    return [lat_era5 + half, lon_era5 - half, lat_era5 -half, lon_era5 + half]


df_pontos = (
    df_locais[["CEG", "lat_base", "lon_base"]]
    .dropna()
    .drop_duplicates(subset="CEG")
    .reset_index(drop=True)
)

df_pontos[["lat_era5", "lon_era5"]] = df_pontos.apply(
    lambda r: pd.Series(snap_to_era5_grid(r["lat_base"], r["lon_base"])),
    axis=1
)

df_pontos = df_pontos.drop_duplicates(subset=["lat_era5", "lon_era5"]).reset_index(drop=True)

print(f"Pontos (células ERA5) únicos para baixar: {len(df_pontos)}")
print(df_pontos.head())

df_parque_cell = (
    df_locais[["CEG", "lat_base", "lon_base"]]
    .dropna()
    .drop_duplicates(subset="CEG")
    .reset_index(drop=True)
)

# Criar cell_key para cada turbina (adicionar APÓS df_parque_cell)
def criar_cell_key(lat_era5, lon_era5):
    lat_str = f"lat{lat_era5:.2f}".replace(".", "p").replace("-", "m")
    lon_str = f"lon{lon_era5:.2f}".replace(".", "p").replace("-", "m")
    return f"{lat_str}_{lon_str}"



df_parque_cell[["lat_era5", "lon_era5"]] = df_parque_cell.apply(
    lambda r: pd.Series(snap_to_era5_grid(r["lat_base"], r["lon_base"])),
    axis=1
)

df_parque_cell["cell_key"] = df_parque_cell.apply(
    lambda r: criar_cell_key(r["lat_era5"], r["lon_era5"]),
    axis=1
)


df_cells = df_parque_cell.drop_duplicates(subset=["cell_key"]).reset_index(drop=True)

# salva o “catálogo” pra pipeline futura
df_parque_cell.to_csv(os.path.join(pasta_era5, "catalogo_parque_para_celula.csv"), index=False)




# Request
c = cdsapi.Client()

vars = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
     "100m_u_component_of_wind",
     "100m_v_component_of_wind",
    "2m_temperature",
    "sea_surface_temperature",
    "surface_pressure",
    "total_precipitation",
    "friction_velocity"
]



lst_errors = []

print("Download Iniciado...")

print(df_cells["cell_key"].nunique())

def sleep_por_erro(e: Exception):
    msg = str(e).lower()

    # mesma lógica que você já tinha
    if "429" in msg or "too many requests" in msg or "rate limit" in msg:
        print("Erro 429 / rate limit. Aguardando 1 hora...")
        time.sleep(3600)
        return

    if "quota" in msg or "limit" in msg:
        print("Cota/limite excedido. Aguardando 30 minutos...")
        time.sleep(1800)
        return

    # erros de rede comuns (opcional: mais leve que 2 minutos)
    if "timeout" in msg or "timed out" in msg or "connection" in msg or "proxy" in msg:
        print("Erro de rede/timeout. Aguardando 5 minutos...")
        time.sleep(300)
        return

    print("Erro genérico. Aguardando 2 minutos antes da próxima tentativa...")
    time.sleep(120)


# Função de download com backoff corrigida
@backoff.on_exception(backoff.expo,
                      Exception,
                      max_tries=3,
                      base=2,
                      max_time=4000)
def retrieve_and_validate(ano: int, mes: str, bbox: list, output_file: str):
    # Calcular dias válidos para o mês/ano
    _, last_day = monthrange(ano, int(mes))
    dias_validos = [str(d).zfill(2) for d in range(1, last_day + 1)]

    c.retrieve('reanalysis-era5-single-levels', {
        'product_type': 'reanalysis',
        'variable': vars,
        'area': bbox,
        'year': str(ano),
        'month': mes,
        'day': dias_validos,
        'time': [f"{h:02d}:00" for h in range(0,24,12)],
        'data_format': 'netcdf'  # Usar exatamente como mostrado no site do CDS
    }, output_file)

def retrieve_year_monthly_means(ano: int, bbox: list, output_file: str):
    c.retrieve(
        "reanalysis-era5-single-levels-monthly-means",{
            "product_type": "monthly_averaged_reanalysis",
            "variable": vars,
            "year": str(ano),
            "month": meses,      # 01..12
            "time": "00:00",
            "area": bbox,
            "format": "netcdf",
            "grid": "0.25/0.25",
        },
        output_file
    )

def process_downloaded_file(file_path):
    """
    Processa arquivo baixado, extraindo ZIP se necessário e fazendo merge
    """
    try:
        # Verifica se é um arquivo ZIP
        if zipfile.is_zipfile(file_path):
            print(f"Arquivo ZIP detectado: {file_path}")

            # Cria diretório temporário para extração
            extract_dir = file_path.replace('.nc', '_temp_extract')
            os.makedirs(extract_dir, exist_ok=True)

            # Extrai o ZIP
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                zip_contents = zip_file.namelist()
                print(f"Conteúdo do ZIP: {zip_contents}")
                zip_file.extractall(extract_dir)

            # Encontra arquivos .nc extraídos
            nc_files = [f for f in zip_contents if f.endswith('.nc')]

            if len(nc_files) == 0:
                raise Exception("Nenhum arquivo NetCDF encontrado no ZIP")
            elif len(nc_files) == 1:
                # Apenas um arquivo - move para o local final
                src_file = os.path.join(extract_dir, nc_files[0])
                # Remove o ZIP original
                os.remove(file_path)
                # Move o arquivo extraído para o local final
                shutil.move(src_file, file_path)
                print(f"Arquivo único extraído: {nc_files[0]}")
            else:
                # Múltiplos arquivos - faz merge
                print(f"Fazendo merge de {len(nc_files)} arquivos NetCDF...")

                datasets = []
                for nc_file in nc_files:
                    nc_path = os.path.join(extract_dir, nc_file)
                    print(f"Carregando: {nc_file}")
                    ds = xr.open_dataset(nc_path)
                    datasets.append(ds)

                # Faz o merge dos datasets
                print("Combinando datasets...")
                try:
                    # Tenta merge primeiro
                    combined = xr.merge(datasets)
                except Exception as e:
                    print(f"Merge falhou, tentando concatenar: {e}")
                    # Se merge falhar, tenta concatenar por time
                    combined = xr.concat(datasets, dim='time')

                # Remove arquivo ZIP original
                os.remove(file_path)

                # Salva o arquivo combinado
                print(f"Salvando arquivo combinado: {file_path}")
                combined.to_netcdf(file_path)

                # Fecha todos os datasets
                for ds in datasets:
                    ds.close()
                combined.close()

                print(f"Merge concluído com sucesso")

            # Limpa diretório temporário
            shutil.rmtree(extract_dir, ignore_errors=True)
            return True

        else:
            # Não é ZIP, arquivo já está no formato correto
            print(f"Arquivo NetCDF direto (não é ZIP): {file_path}")
            return True

    except Exception as e:
        print(f"Erro ao processar arquivo {file_path}: {e}")
        # Limpa arquivos temporários em caso de erro
        if 'extract_dir' in locals() and os.path.exists(extract_dir):
            shutil.rmtree(extract_dir, ignore_errors=True)
        return False


def validate_netcdf(file_path):
    """Valida se o arquivo NetCDF foi baixado corretamente"""
    try:
        with xr.open_dataset(file_path) as ds:
            # Verifica se contém pelo menos algumas variáveis esperadas
            expected_vars = ['u10', 'v10', 't2m', 'sp', 'tp']  # Variáveis mais comuns
            found_vars = [var for var in expected_vars if var in ds.variables]

            if len(found_vars) == 0:
                print(f"Nenhuma variável esperada encontrada no arquivo")
                print(f"Variáveis disponíveis: {list(ds.data_vars)}")
                return False

            print(f"Arquivo válido. Dimensões: {dict(ds.sizes)}")
            print(f"Variáveis encontradas: {found_vars}")
            return True
    except Exception as e:
        print(f"Erro ao validar arquivo {file_path}: {e}")
        return False


for _, p in df_cells.iterrows():
    bbox = bbox_single_cell(p["lat_era5"], p["lon_era5"])

    point_key = p["cell_key"]

    for ano in anos:
        pasta_ano = os.path.join(pasta_era5, f"ano={ano}")
        os.makedirs(pasta_ano, exist_ok=True)

        nome_arq= f"Era5M_NE-{point_key}_{ano}.nc"
        output_file= os.path.join(pasta_ano, nome_arq)

        if os.path.exists(output_file):
            print(f"Já existe: {nome_arq} -> pulando")
            if validate_netcdf(output_file):
                continue
            else:
                print("Arquivo existente corrompido. Removendo...")
                os.remove(output_file)


        print(f"Baixando MONTHLY {point_key} | ano={ano} | bbox={bbox}")

        ok = False
        tentativas = 0
        while not ok:
            tentativas +=1
            try:
        
                retrieve_year_monthly_means(
                    ano=ano,
                    bbox=bbox,
                    output_file=output_file)

                if not process_downloaded_file(output_file):
                    raise Exception("Erro ao processar (zip/merge)")

                if not validate_netcdf(output_file):
                    raise Exception("Arquivo inválido após download")
                
                ok=True

            except Exception as e:
                print(f"[ERRO] {point_key} ano={ano}: {e}")
                lst_errors.append((point_key, ano, str(e)))
                if os.path.exists(output_file):
                    os.remove(output_file)
                
                sleep_por_erro(e)

                if tentativas >= 8:
                    print("MAIS DE 8 TENTATIVAS AHHHH")
                    break



print("\nTodos os downloads concluídos!")
if lst_errors:
    print(f"Erros encontrados: {len(lst_errors)}")
    for erro in lst_errors:
        print(f"  {erro[0]}-{erro[1]}: {erro[2]}")
else:
    print("Nenhum erro encontrado!")

# Exemplo de como ler os arquivos após download
def ler_arquivo_era5(caminho_arquivo):
    """Função para ler arquivos ERA5 baixados"""
    try:
        ds = xr.open_dataset(caminho_arquivo)
        print(f"Arquivo carregado: {caminho_arquivo}")
        print(f"Variáveis disponíveis: {list(ds.variables)}")
        # Verifica qual dimensão temporal está disponível
        if 'time' in ds.dims:
            time_var = 'time'
        elif 'valid_time' in ds.dims:
            time_var = 'valid_time'
        else:
            time_var = None

        if time_var:
            print(f"Período: {ds[time_var].min().values} a {ds[time_var].max().values}")
        else:
            print("Dimensão temporal não encontrada")
        return ds
    except Exception as e:
        print(f"Erro ao ler arquivo {caminho_arquivo}: {e}")
        return None

# Exemplo de uso da função de leitura
print("\n--- Testando leitura dos arquivos baixados ---")
for root, _, files in os.walk(pasta_era5):
    for arquivo in files:
        if arquivo.endswith('.nc'):
            caminho_completo = os.path.join(root, arquivo)
            ds = ler_arquivo_era5(caminho_completo)
            if ds is not None:
                ds.close()  # Importante: fechar o dataset após uso

netCDF4 disponível
Pontos (células ERA5) únicos para baixar: 225
                  CEG   lat_base   lon_base  lat_era5  lon_era5
0  EOLCVBA030751-3-01 -14.397389 -42.594263    -14.50    -42.50
1  EOLCVPE030806-4-01  -9.060335 -38.141453     -9.00    -38.25
2  EOLCVRN030549-9-01  -5.049454 -35.970497     -5.00    -36.00
3  EOLCVBA030727-0-01 -11.817059 -41.400365    -11.75    -41.50
4  EOLCVRN030834-0-01  -5.443699 -35.925087     -5.50    -36.00
Download Iniciado...
225
Baixando MONTHLY latm14p50_lonm42p50 | ano=2015 | bbox=[-14.375, -42.625, -14.625, -42.375]


2026-01-07 12:03:40,243 INFO Request ID is 8d02f9ae-a9df-4d09-b129-df6a1963e1e6
2026-01-07 12:03:40,510 INFO status has been updated to accepted
2026-01-07 13:24:47,598 INFO status has been updated to running
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/retrieve/v1/jobs/8d02f9ae-a9df-4d09-b129-df6a1963e1e6?log=True&request=True (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021AAFFE3610>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11002] getaddrinfo failed)"))], attempt 1 of 500
Retrying in 120 seconds
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/retrieve/v1/jobs/8d02f9ae-a9df-4d09-b129-df6a1963e1e6?log=True&request=True (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021AAFDD0410>: Failed to resolve 'cds.climate.copernicus.eu

KeyboardInterrupt: 

In [ ]:
#Tratando dados do ERA5 + Cálculo de Potência
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
import os
import re
import gc
import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path
from glob import glob
from tqdm import tqdm

# Configurações

master_path = dataset_locais
output_path = serie_temporal
os.makedirs(output_path, exist_ok=True)

# Constantes físicas
R_ESPECIFICO_AR_SECO = 287.058  # J/(kg·K)

# Constantes de turbina para cálculo de potência
CUT_IN, RATED, CUT_OUT = 3.0, 12.0, 25.0
FATOR_PERDAS = 0.85

lst_errors = []

print("Carregando coordenadas das turbinas...")
df_turbinas = pd.read_csv(master_path)

df_turbinas["lat_use"] = df_turbinas["latitude_turbina"].fillna(df_turbinas["latitude_parque"])
df_turbinas["lon_use"] = df_turbinas["longitude_turbina"].fillna(df_turbinas["longitude_parque"])
df_turbinas = df_turbinas.dropna(subset=["lat_use", "lon_use"])


# Para parques sem turbinas específicas, usa coordenadas do parque
mask_sem_turbina = df_turbinas['denominacao_aerogerador'].isna()
if mask_sem_turbina.any():
    print(f"Encontrados {mask_sem_turbina.sum()} parques sem turbinas específicas - usando coordenadas do parque")
    df_turbinas.loc[mask_sem_turbina, 'denominacao_aerogerador'] = df_turbinas.loc[mask_sem_turbina, 'CEG']

# --- cria cell_key compatível com o nome do arquivo ERA5_NE-lat..._lon..._YYYY-MM.nc ---
RES = 0.25
def snap_to_era5_grid(lat, lon, res=RES):
    return round(lat/res)*res, round(lon/res)*res

def criar_cell_key(lat_era5, lon_era5):
    lat_str = f"lat{lat_era5:.2f}".replace(".", "p").replace("-", "m")
    lon_str = f"lon{lon_era5:.2f}".replace(".", "p").replace("-", "m")
    return f"{lat_str}_{lon_str}"

def parse_cell_key(cell_key: str):
    # cell_key: "latm14p50_lonm42p50"
    lat_part, lon_part = cell_key.split("_")
    lat = float(lat_part.replace("lat", "").replace("m", "-").replace("p", "."))
    lon = float(lon_part.replace("lon", "").replace("m", "-").replace("p", "."))
    return lat, lon


df_turbinas[["lat_era5", "lon_era5"]] = df_turbinas.apply(
    lambda r: pd.Series(snap_to_era5_grid(r["lat_use"], r["lon_use"])),
    axis=1
)
df_turbinas["cell_key"] = df_turbinas.apply(
    lambda r: criar_cell_key(r["lat_era5"], r["lon_era5"]),
    axis=1
)

print(f"Total de pontos para extração: {len(df_turbinas)}")

arquivos_era5 = [
    f for f in glob(os.path.join(pasta_era5, "ano=*", "Era5M_NE-*.nc"), recursive=True)
    if "_temp" not in f
]

print(f"Arquivos ERA5 encontrados: {len(arquivos_era5)}")
if len(arquivos_era5) > 0:
    print("Exemplo:", arquivos_era5[0])



def extrair_dados_pontos(ds, lat, lon):
    """Extrai dados para um ponto usando interpolação"""
    try:
        ponto = ds.sel(latitude=lat, longitude=lon, method='nearest')
        ponto_df = ponto.to_dataframe().reset_index()
        return ponto_df
    except Exception as e:
        print(f"Erro ao extrair dados para ponto ({lat}, {lon}): {e}")
        return None

def calcular_potencia_gerada(velocidade_vento, densidade_ar, diametro_rotor, potencia_nominal, ano, ano_inicio_operacao):
    """Calcula potência gerada baseada na curva de potência simplificada"""
    if pd.isna(velocidade_vento) or pd.isna(densidade_ar) or pd.isna(diametro_rotor) or pd.isna(potencia_nominal):
        return 0.0
    if ano < ano_inicio_operacao:
      return 0.0

    if velocidade_vento >= RATED and velocidade_vento < CUT_OUT:
        # Região de potência nominal
        return potencia_nominal * FATOR_PERDAS
    elif velocidade_vento >= CUT_IN and velocidade_vento < RATED:
        # Região de potência variável (limitada pela nominal)
        area_rotor = np.pi * (diametro_rotor / 2) ** 2
        potencia_teorica = (0.5 * densidade_ar * (velocidade_vento ** 3) * area_rotor) / 1e6  # MW
        return min(potencia_teorica, potencia_nominal) * FATOR_PERDAS
    else:
        # Abaixo do cut-in ou acima do cut-out
        return 0.0

def processar_arquivo_era5(arquivo_nc, df_turbinas):
    """Processa arquivo ERA5 e seu par _temp.nc"""
    print(f"Processando: {os.path.basename(arquivo_nc)}")

    # --- INÍCIO DO TRECHO DE JUNÇÃO ---
    arquivo_principal = arquivo_nc
    arquivo_temp = arquivo_principal.replace('.nc', '_temp.nc')

    # Verifica se ambos os arquivos existem
    if not os.path.exists(arquivo_temp):
        #print(f"[WARN] Arquivo complementar {os.path.basename(arquivo_temp)} não encontrado. Seguindo sem concatenar.")
        arquivo_temp = None

    # Abre os dois datasets e os junta
    with xr.open_dataset(arquivo_principal) as ds_main:
        if (arquivo_temp is not None):
            with xr.open_dataset(arquivo_temp) as ds_temp:
                ds_completo = xr.merge([ds_main, ds_temp])
        else:
            ds_completo = ds_main

        # Padroniza dimensão temporal
        if 'valid_time' in ds_completo.dims:
            ds_completo = ds_completo.rename({'valid_time': 'time'})

        dados_extraidos = []

        for idx, turbina in tqdm(df_turbinas.iterrows(), total=len(df_turbinas),
                                desc="Extraindo dados das turbinas"):
            lat = turbina['lat_use']
            lon = turbina['lon_use']

            ponto_df = extrair_dados_pontos(ds_completo, lat, lon)

            # --- COLAR LOGO APÓS extrair_dados_pontos(...) e ANTES das engenharias ---
            ponto_df["time"] = pd.to_datetime(ponto_df["time"])
            
            # Agrupa por mês e consolida as variáveis (resolve o esquema 00:00 vs 06:00)
            ponto_df["time_mes"] = ponto_df["time"].dt.to_period("M").dt.to_timestamp()
            
            # Mantém 1 linha por mês (mean ignora NaN, então junta corretamente as colunas)
            ponto_df = (
                ponto_df
                .drop(columns=["time"])
                .groupby(["time_mes"], as_index=False)
                .mean(numeric_only=True)
                .rename(columns={"time_mes": "time"})
            )
            # --- FIM DO BLOCO ---



            if ponto_df is not None and not ponto_df.empty:

                ponto_df['CEG'] = turbina['CEG']
                ponto_df['denominacao_aerogerador'] = turbina['denominacao_aerogerador']

                # === ADICIONA DADOS TÉCNICOS DA TURBINA ===
                ponto_df['altura_torre_m'] = turbina['altura_torre_m']
                ponto_df['diametro_rotor_m'] = turbina['diametro_rotor_m']
                ponto_df['potencia_turbina_mw'] = turbina['potencia_turbina_mw']
                ponto_df['nome_parque'] = turbina['nome_parque']
                ponto_df['status_operacao'] = turbina['status_operacao']
                ponto_df['ano_inicio_operacao'] = turbina['ano_inicio_operacao']

                # Engenharia de variáveis básica
                # 1. Calcular Velocidade e Direção do Vento
                ponto_df['velocidade_vento_100m'] = np.sqrt(ponto_df['u100']**2 + ponto_df['v100']**2)
                ponto_df['velocidade_vento_10m'] = np.sqrt(ponto_df['u10']**2 + ponto_df['v10']**2)
                # Direção em radianos, depois convertida para graus (0-360)
                ponto_df['direcao_vento_100m'] = np.mod(180 + np.rad2deg(np.arctan2(ponto_df['u100'], ponto_df['v100'])), 360)
                ponto_df['densidade_ar'] = ponto_df['sp'] / (R_ESPECIFICO_AR_SECO * ponto_df['t2m'])

                # 2. Calcular Wind Shear Exponent (alpha)
                # Evita divisão por zero se o vento a 10m for nulo
                # a = ln(v2/v1) / ln(h2/h1) -> ln(100/10) = 2.3025
                ponto_df['wind_shear'] = np.log(ponto_df['velocidade_vento_100m'] / ponto_df['velocidade_vento_10m'].replace(0, 0.01)) / 2.3025
                ponto_df['wind_shear'] = ponto_df['wind_shear'].fillna(0) # Preenche casos onde a velocidade é zero

                # 3. Calcular Intensidade da Turbulência (proxy)
                if "zust" in ponto_df.columns:
                    ponto_df['intensidade_turbulencia'] = ponto_df['zust'] / ponto_df['velocidade_vento_100m'].replace(0, 0.01)
                    ponto_df['intensidade_turbulencia'] = ponto_df['intensidade_turbulencia'].fillna(0)
                else:
                    ponto_df["intensidade_turbulencia"] = 0.0

                ponto_df['potencia_vento_100m'] = 0.5 * ponto_df['densidade_ar'] * (ponto_df['velocidade_vento_100m']**3)

                # 4. Criar Variáveis Temporais
                ponto_df['time'] = pd.to_datetime(ponto_df['time'])
                ponto_df['hora_do_dia'] = ponto_df['time'].dt.hour
                ponto_df['dia_do_ano'] = ponto_df['time'].dt.dayofyear
                ponto_df['mes'] = ponto_df['time'].dt.month

                # === ADICIONA VARIÁVEIS TEMPORAIS EXTRAS PARA PARTICIONAMENTO ===
                ponto_df['ano'] = ponto_df['time'].dt.year
                ponto_df['dia'] = ponto_df['time'].dt.day

                # === CÁLCULO DA POTÊNCIA GERADA ESTIMADA ===
                ponto_df['potencia_gerada_estimada_mw'] = ponto_df.apply(
                    lambda row: calcular_potencia_gerada(
                        row['velocidade_vento_100m'],
                        row['densidade_ar'],
                        row['diametro_rotor_m'],
                        row['potencia_turbina_mw'],
                        row['ano'],
                        row['ano_inicio_operacao']
                    ), axis=1
                )

                dados_extraidos.append(ponto_df)

            else:
                lst_errors.append(arquivo_nc)

        if dados_extraidos:
            df_completo = pd.concat(dados_extraidos, ignore_index=True)
            return df_completo
        else:
            return None
        #Fim do trecho de junção ---

# Processa todos os arquivos (usando append incremental)
print("Iniciando extração de dados...")

lst_errors_arqs = []

for i, arquivo in enumerate(arquivos_era5):
    stem = Path(arquivo).stem  # ex: ERA5_NE-latm9p0_lonm38p25_2024-06
    m = re.match(r"Era5M_NE-(lat[^_]+_lon[^_]+)_(\d{4})$", stem)

    if not m:
        lst_errors_arqs.append(arquivo)
        raise ValueError(f"Nome inesperado: {stem}")
        continue
    
    cell_key, ano = m.group(1), m.group(2)

    lat_raw, lon_raw = parse_cell_key(cell_key)
    lat_era5, lon_era5 = snap_to_era5_grid(lat_raw, lon_raw)


    
    
    nome_base = f"{cell_key}_{ano}"

    output_file = Path(output_path) / f"series_temporais_{nome_base}.parquet"

    if os.path.exists(output_file):
        print(f"Arquivo series_temporais_{nome_base} já existe. Pulando processamento.")
        continue

    df_turb_cell = df_turbinas[df_turbinas["cell_key"] == cell_key]
    if df_turb_cell.empty:
        continue

    df_arquivo = processar_arquivo_era5(arquivo, df_turb_cell)

    if df_arquivo is None or df_arquivo.empty:
      lst_errors_arqs.append(arquivo)
      continue
      # Primeiro arquivo: cria novo parquet


    df_arquivo.to_parquet(output_file, index=False, compression="snappy")
    del df_arquivo   # remove a referência
    gc.collect()


print("Arquivos com erro:", lst_errors)
print("Arquivos não gerados (nulos):", lst_errors_arqs)
print("Extração concluída!")

# Verifica resultado final

parquet_files = glob(os.path.join(output_path, "series_temporais_*.parquet"))

if parquet_files:
    print(f"\nForam gerados {len(parquet_files)} arquivos parquet.")

    # ---- acumuladores ----
    total_registros      = 0
    periodos_temporais   = set()
    turbinas_unicas      = set()
    stats_potencia       = []

    # percorre arquivo a arquivo
    for f in parquet_files:
        cols = [
            "time",
            "denominacao_aerogerador",
            "potencia_gerada_estimada_mw"
        ]  # só o que precisamos para as métricas

        df = pd.read_parquet(f, columns=cols)

        total_registros    += len(df)
        periodos_temporais |= set(df["time"].unique())
        turbinas_unicas    |= set(df["denominacao_aerogerador"].unique())
        stats_potencia.append(df["potencia_gerada_estimada_mw"])

    # ---- métricas gerais ----
    all_pot = pd.concat(stats_potencia, ignore_index=True)

    print("\nDados extraídos:")
    print(f"- Total de registros: {total_registros:,}")
    print(f"- Períodos temporais: {len(periodos_temporais)}")
    print(f"- Turbinas/pontos processados: {len(turbinas_unicas)}")
    # número de colunas é igual em todos — basta pegar do último df
    print(f"- Variáveis disponíveis: {len(df.columns)} colunas")

    if not all_pot.empty:
        print("\nEstatísticas da Potência Gerada:")
        print(f"- Potência média: {all_pot.mean():.3f} MW")
        print(f"- Potência máxima: {all_pot.max():.3f} MW")
        print(f"- % tempo gerando: {(all_pot > 0).mean()*100:.1f}%")

    # ── cria pivot para ML (amostra ou completo) ──
    variaveis_para_pivotar = [
        'velocidade_vento_100m',
        'densidade_ar',
        'potencia_vento_100m',
        'direcao_vento_100m',
        'wind_shear',
        'intensidade_turbulencia',
        'potencia_gerada_estimada_mw',
        'tp'
    ]

    # Lê de novo somente as colunas de interesse (podem ser grandes!)
    df_pivot_iter = []
    for f in parquet_files:
        df_tmp = pd.read_parquet(f, columns=["time", "denominacao_aerogerador", *variaveis_para_pivotar])
        df_pivot_iter.append(df_tmp)

    df_final_pivot = pd.concat(df_pivot_iter, ignore_index=True)

    df_pivotado = (
        df_final_pivot
        .pivot_table(
            index='time',
            columns='denominacao_aerogerador',
            values=variaveis_para_pivotar,
            aggfunc='mean'
        )
        .reset_index()
    )
    df_pivotado.columns = ['_'.join(col).strip() for col in df_pivotado.columns.values]
    output_pivot = os.path.join(output_path, 'series_temporais_pivotado.parquet')
    df_pivotado.to_parquet(output_pivot, index=False)
    print(f"\nDataset pivotado salvo: {df_pivotado.shape[0]} linhas × {df_pivotado.shape[1]-1} colunas")
    print(f"Arquivos salvos em: {output_path}")

else:
    print("Nenhum dado foi extraído. Verifique os arquivos ERA5.")

Carregando coordenadas das turbinas...
Encontrados 587 parques sem turbinas específicas - usando coordenadas do parque
Total de pontos para extração: 17758
Arquivos ERA5 encontrados: 54
Exemplo: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\ERA5_Bruto\latm11p598538859999962_lonm41p339425792999975\ERA5_NE-latm11p598538859999962_lonm41p339425792999975_2024-06.nc
Iniciando extração de dados...
Processando: ERA5_NE-latm11p598538859999962_lonm41p339425792999975_2024-06.nc


Extraindo dados das turbinas: 100%|██████████| 226/226 [00:04<00:00, 53.10it/s]


Processando: ERA5_NE-latm11p66424999999998_lonm41p361999999999966_2024-06.nc


Extraindo dados das turbinas: 100%|██████████| 40/40 [00:00<00:00, 48.13it/s]


Processando: ERA5_NE-latm11p796463157999938_lonm41p39338419699993_2024-06.nc


...

Processando: ERA5_NE-latm9p498583332999942_lonm40p87899999999996_2024-06.nc


Extraindo dados das turbinas: 100%|██████████| 66/66 [00:10<00:00,  6.04it/s]


Arquivo series_temporais_latm9p5_lonm41p0_2024-06 já existe. Pulando processamento.
Processando: ERA5_NE-latm9p863843840999948_lonm41p10417463999994_2024-06.nc


Extraindo dados das turbinas: 100%|██████████| 173/173 [00:10<00:00, 16.82it/s]


Arquivo series_temporais_latm9p75_lonm41p0_2024-06 já existe. Pulando processamento.
Arquivo series_temporais_latm9p75_lonm41p0_2024-06 já existe. Pulando processamento.
Arquivos com erro: []
Arquivos não gerados (nulos): []
Extração concluída!

Foram gerados 26 arquivos parquet.

Dados extraídos:
- Total de registros: 3,404,160
- Períodos temporais: 720
- Turbinas/pontos processados: 3843
- Variáveis disponíveis: 3 colunas

Estatísticas da Potência Gerada:
- Potência média: 1.866 MW
- Potência máxima: 5.610 MW
- % tempo gerando: 97.7%

Dataset pivotado salvo: 720 linhas × 30744 colunas
Arquivos salvos em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\Series_Temporais


# Vizualização e Tratamento dos dados
***Preparação para Input nos MLs***

In [ ]:
import os
from pathlib import Path
import polars as pl
import pyarrow.parquet as pa_parquet
import pyarrow.dataset as pa_ds
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import gc
import matplotlib.pyplot as plt
import seaborn as sns

def run_eda_enriquecida(n_bins: int = 50):
    """EDA completa processando arquivos em lotes para evitar estouro de memória"""

    # ---- entradas já existentes no seu cabeçalho ----
    serie_temporal_path = Path(serie_temporal)
    eda_dir = Path(EDA_DIR); eda_dir.mkdir(parents=True, exist_ok=True)
    pattern = str(serie_temporal_path / "series_temporais_*.parquet")
    time_col = "time"
    high_card_ids = ["CEG", "denominacao_aerogerador", "nome_parque"]

    # Lista de arquivos parquet
    PARQUETS = sorted(serie_temporal_path.glob("series_temporais_*.parquet"))

    # ---- helpers ----
    NUMERIC_DTYPES = {
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64,
    }

    def is_numeric_dtype(dt: pl.DataType) -> bool:
        return dt in NUMERIC_DTYPES

    def to_iso(x):
        return pd.Timestamp(x).isoformat()

    def monitor_memory():
        """Monitora uso de memória durante processamento"""
        try:
            import psutil
            process = psutil.Process()
            memory_mb = process.memory_info().rss / 1024 / 1024
            print(f"Uso de memória: {memory_mb:.1f} MB")
            return memory_mb
        except ImportError:
            print("psutil não instalado")
            return 0

    # ---- schema (compatível entre versões do Polars) ----
    lf_all = pl.scan_parquet(pattern)
    try:
        schema = lf_all.collect_schema()
    except AttributeError:
        schema = lf_all.schema

    # Remover coluna sst (única coluna com muitos nulos e não importante para análise)
    all_cols = [c for c in schema.keys() if c not in high_card_ids and c != time_col and c != "sst"]
    num_cols = [c for c in all_cols if is_numeric_dtype(schema[c])]

    print(f"Colunas após remoção de sst: {all_cols}")
    print(f"Colunas numéricas: {num_cols}")

    print(f"Processando {len(PARQUETS)} arquivos parquet...")
    print(f"Colunas numéricas encontradas: {len(num_cols)}")
    monitor_memory()

    # Salvar schema
    with open(eda_dir / "schema.txt", "w", encoding="utf-8") as f:
        for col, dtype in schema.items():
            if col != "sst":  # Não incluir sst no schema salvo
                f.write(f"{col}: {dtype}\n")

    # =========================
    # 0) Cálculo de percentis globais para clipping (arquivo por arquivo)
    # =========================
    print("\n0) Calculando percentis globais para clipping de outliers...")

    # Definir estratégias de clipping por variável
    PERCENTILE_CLIPPING_COLS = ["wind_shear", "intensidade_turbulencia"]  # Clipping por percentis

    FIXED_CLIPPING_COLS = {
        # Variáveis com limites físicos conhecidos
        "t2m": {"min": 200.0, "max": 330.0},           # Temperatura (K)
        "sp": {"min": 20000.0, "max": 120000.0},       # Pressão superficial (Pa)
        "densidade_ar": {"min": 0.5, "max": 1.6}       # Densidade do ar (kg/m³)
    }

    # Filtrar colunas que existem no dataset
    percentile_cols = [c for c in PERCENTILE_CLIPPING_COLS if c in num_cols]
    fixed_cols = {k: v for k, v in FIXED_CLIPPING_COLS.items() if k in num_cols}

    # Calcular percentis P1/P99 arquivo por arquivo (baixo consumo de memória)
    percentiles_global = {}

    if percentile_cols:
        print(f"Calculando percentis para {len(percentile_cols)} colunas: {percentile_cols}")

        # Inicializar acumuladores de percentis
        percentile_acc = {col: [] for col in percentile_cols}

        for file in tqdm(PARQUETS, desc="Coletando amostras para percentis"):
            try:
                df = pl.read_parquet(file).select(percentile_cols).drop_nulls()
                if len(df) > 0:
                    for col in percentile_cols:
                        if col in df.columns:
                            # Coletar amostra representativa (cada 10ª linha para economizar memória)
                            sample = df[col][::10].to_list()
                            percentile_acc[col].extend(sample)
            except Exception as e:
                print(f"Erro coletando amostras de {file.name}: {e}")
                continue

            # Limpeza de memória
            del df
            gc.collect()

        # Calcular percentis finais
        for col in percentile_cols:
            if percentile_acc[col]:
                values = percentile_acc[col]
                p1 = np.percentile(values, 1)
                p99 = np.percentile(values, 99)
                percentiles_global[col] = {"p1": p1, "p99": p99, "type": "percentile"}
                print(f"{col}: P1={p1:.3f}, P99={p99:.3f}")
            else:
                percentiles_global[col] = {"p1": 0, "p99": 0, "type": "percentile"}

        # Limpar acumuladores da memória
        del percentile_acc
        gc.collect()

    # Adicionar limites físicos fixos
    for col, limits in fixed_cols.items():
        percentiles_global[col] = {
            "p1": limits["min"],
            "p99": limits["max"],
            "type": "fixed_physical"
        }
        print(f"{col}: Limites físicos min={limits['min']}, max={limits['max']}")

    def apply_outlier_clipping(df: pl.DataFrame) -> pl.DataFrame:
        """Aplica clipping de outliers baseado em percentis ou limites físicos"""
        if not percentiles_global:
            return df

        exprs = []
        for col in df.columns:
            if col in percentiles_global:
                p1 = percentiles_global[col]["p1"]
                p99 = percentiles_global[col]["p99"]
                clipping_type = percentiles_global[col]["type"]

                if clipping_type == "percentile":
                    # Para wind_shear e intensidade_turbulencia:
                    # - Se valor > p99 (outlier extremo), substitui por 0 (indica vento muito baixo)
                    # - Se valor < p1, substitui por p1
                    expr = (
                        pl.when(pl.col(col) > p99)
                        .then(0.0)  # Outliers extremos = 0
                        .when(pl.col(col) < p1)
                        .then(p1)   # Outliers inferiores = p1
                        .otherwise(pl.col(col))
                        .alias(col)
                    )
                else:  # fixed_physical
                    # Para variáveis com limites físicos: clippar nos limites
                    expr = (
                        pl.col(col).clip(p1, p99).alias(col)
                    )

                exprs.append(expr)
            else:
                exprs.append(pl.col(col))

        return df.with_columns(exprs)

    # =========================
    # 1) Estatísticas mensais (count/mean/std/min/max) - COM CLIPPING
    # =========================
    print("\n1) Calculando estatísticas mensais (com clipping de outliers)...")
    monthly_acc = defaultdict(lambda: defaultdict(lambda: {"cnt":0,"sum":0.0,"sumsq":0.0,"min":None,"max":None}))

    for file in tqdm(PARQUETS, desc="Estatísticas mensais"):
        try:
            # Ler, clippar e processar arquivo por arquivo
            df = pl.read_parquet(file)
            if "sst" in df.columns:
                df = df.drop("sst")  # Remover sst

            df_clipped = apply_outlier_clipping(df)

            lff = (
                df_clipped.lazy()
                  .with_columns(pl.col(time_col).cast(pl.Datetime))
                  .sort(time_col)
                  .group_by_dynamic(index_column=time_col, every="1mo")
            )
            aggs = []
            for c in num_cols:
                if c in df_clipped.columns:
                    aggs.extend([
                        pl.col(c).count().alias(f"{c}__cnt"),
                        pl.col(c).cast(pl.Float64).sum().alias(f"{c}__sum"),  # Float64 para evitar overflow
                        (pl.col(c).cast(pl.Float64) * pl.col(c).cast(pl.Float64)).sum().alias(f"{c}__sumsq"),
                        pl.col(c).min().alias(f"{c}__min"),
                        pl.col(c).max().alias(f"{c}__max"),
                    ])

            if aggs:
                rows = (lff.agg(aggs)
                          .rename({time_col: "mes"})
                          .collect()
                          .to_dicts())

                for r in rows:
                    mes = to_iso(r["mes"])
                    for c in num_cols:
                        if c in df_clipped.columns:
                            a = monthly_acc[mes][c]
                            a["cnt"] += int(r.get(f"{c}__cnt", 0) or 0)
                            a["sum"] += float(r.get(f"{c}__sum", 0.0) or 0.0)
                            a["sumsq"] += float(r.get(f"{c}__sumsq", 0.0) or 0.0)
                            mn = r.get(f"{c}__min", None)
                            mx = r.get(f"{c}__max", None)
                            a["min"] = mn if a["min"] is None else (mn if mn is not None and mn < a["min"] else a["min"])
                            a["max"] = mx if a["max"] is None else (mx if mx is not None and mx > a["max"] else a["max"])

        except Exception as e:
            print(f"Erro processando estatísticas mensais em {file.name}: {e}")
            continue

        # Limpeza de memória por arquivo
        del df, df_clipped
        gc.collect()

    rows_stats = []
    for mes, d in sorted(monthly_acc.items(), key=lambda x: x[0]):
        for c, a in d.items():
            cnt = a["cnt"]
            if cnt > 0:
                mean = a["sum"] / cnt
                var = max(a["sumsq"]/cnt - mean*mean, 0.0)
                std = np.sqrt(var)  # Desvio padrão populacional
            else:
                mean = std = 0.0
            rows_stats.append({
                "mes": mes, "coluna": c,
                "count": a["cnt"], "mean": mean, "std": std,
                "min": a["min"], "max": a["max"]
            })
    pd.DataFrame(rows_stats).to_csv(eda_dir / "estatisticas_mensais.csv", index=False)

    # =========================
    # 2) Estatísticas globais (count/mean/std/min/max) - COM CLIPPING
    # =========================
    print("\n2) Calculando estatísticas globais (com clipping de outliers)...")
    global_acc = {c: {"cnt":0,"sum":0.0,"sumsq":0.0,"min":None,"max":None} for c in num_cols}

    for file in tqdm(PARQUETS, desc="Estatísticas globais"):
        try:
            # Ler, clippar e processar arquivo por arquivo
            df = pl.read_parquet(file)
            if "sst" in df.columns:
                df = df.drop("sst")  # Remover sst

            df_clipped = apply_outlier_clipping(df)
            lff = df_clipped.lazy()

            aggs = []
            for c in num_cols:
                if c in df_clipped.columns:
                    aggs.extend([
                        pl.col(c).count().alias(f"{c}__cnt"),
                        pl.col(c).cast(pl.Float64).sum().alias(f"{c}__sum"),  # Float64 para evitar overflow
                        (pl.col(c).cast(pl.Float64)*pl.col(c).cast(pl.Float64)).sum().alias(f"{c}__sumsq"),
                        pl.col(c).min().alias(f"{c}__min"),
                        pl.col(c).max().alias(f"{c}__max"),
                    ])

            if aggs:
                r = lff.select(aggs).collect().to_dicts()[0]
                for c in num_cols:
                    if c in df_clipped.columns and c in global_acc:
                        a = global_acc[c]
                        a["cnt"] += int(r.get(f"{c}__cnt", 0) or 0)
                        a["sum"] += float(r.get(f"{c}__sum", 0.0) or 0.0)
                        a["sumsq"] += float(r.get(f"{c}__sumsq", 0.0) or 0.0)
                        mn = r.get(f"{c}__min", None)
                        mx = r.get(f"{c}__max", None)
                        a["min"] = mn if a["min"] is None else (mn if mn is not None and mn < a["min"] else a["min"])
                        a["max"] = mx if a["max"] is None else (mx if mx is not None and mx > a["max"] else a["max"])

        except Exception as e:
            print(f"Erro processando estatísticas globais em {file.name}: {e}")
            continue

        # Limpeza de memória por arquivo
        del df, df_clipped
        gc.collect()

    g_rows = []
    for c, a in global_acc.items():
        cnt = a["cnt"]
        if cnt > 0:
            mean = a["sum"] / cnt
            var = max(a["sumsq"]/cnt - mean*mean, 0.0)
            std = np.sqrt(var)  # Desvio padrão populacional
        else:
            mean = std = 0.0
        g_rows.append({"coluna": c, "count": a["cnt"], "mean": mean, "std": std, "min": a["min"], "max": a["max"]})
    pd.DataFrame(g_rows).to_csv(eda_dir / "estatisticas_globais.csv", index=False)

    # =========================
    # 3) Histogramas globais (n_bins) - COM CLIPPING
    # =========================
    print(f"\n3) Calculando histogramas ({n_bins} bins) com clipping de outliers...")
    gdf = pd.DataFrame(g_rows).set_index("coluna")
    hist_rows = []
    eps = 1e-12

    for c in tqdm([col for col in num_cols if col in gdf.index], desc="Histogramas globais"):
        try:
            cmin = gdf.loc[c, "min"]
            cmax = gdf.loc[c, "max"]
            if pd.isna(cmin) or pd.isna(cmax) or cmin == cmax:
                # coluna constante ou vazia
                hist_rows.append({"coluna": c, "bin": 0, "bin_start": cmin, "bin_end": cmax, "count": int(gdf.loc[c,"count"])})
                continue

            width = (cmax - cmin) / n_bins
            bin_counts = np.zeros(n_bins, dtype=np.int64)

            for file in PARQUETS:
                try:
                    # Ler, clippar e processar histograma arquivo por arquivo
                    df = pl.read_parquet(file)
                    if "sst" in df.columns:
                        df = df.drop("sst")  # Remover sst

                    df_clipped = apply_outlier_clipping(df)
                    lff = df_clipped.lazy()

                    if c not in df_clipped.columns:
                        continue

                    # índice de bin 0..n_bins-1
                    bin_idx = (
                        ((pl.col(c) - cmin) / (cmax - cmin + eps) * n_bins)
                        .floor().clip(0, n_bins-1).cast(pl.Int32).alias("bin")
                    )
                    # agrupa por bin (ignorando nulos)
                    per = (lff.select(bin_idx)
                             .drop_nulls()
                             .group_by("bin")
                             .len()
                             .collect())

                    if per.height:
                        b = per["bin"].to_numpy()
                        v = per["len"].to_numpy()
                        bin_counts[b] += v

                    # Limpeza de memória
                    del df, df_clipped
                    gc.collect()

                except Exception:
                    continue  # coluna pode não existir em algum arquivo

            # escreve linhas desse histograma
            edges = [cmin + i*width for i in range(n_bins+1)]
            for i in range(n_bins):
                hist_rows.append({
                    "coluna": c,
                    "bin": i,
                    "bin_start": edges[i],
                    "bin_end": edges[i+1],
                    "count": int(bin_counts[i])
                })

        except Exception as e:
            print(f"Erro calculando histograma para {c}: {e}")
            continue

    pd.DataFrame(hist_rows).to_csv(eda_dir / f"histogramas_{n_bins}bins.csv", index=False)

    # =========================
    # 4) Correlação (TODOS os dados, não amostral) - COM CLIPPING
    # =========================
    print("\n4) Calculando matriz de correlação (populacional) com clipping de outliers...")

    # Acumuladores para correlação populacional usando método de Welford
    corr_acc = defaultdict(lambda: defaultdict(lambda: {"n": 0, "sum_x": 0.0, "sum_y": 0.0, "sum_xx": 0.0, "sum_yy": 0.0, "sum_xy": 0.0}))

    for file in tqdm(PARQUETS, desc="Calculando correlação populacional"):
        try:
            # Ler, clippar e processar correlação arquivo por arquivo
            df = pl.read_parquet(file)
            if "sst" in df.columns:
                df = df.drop("sst")  # Remover sst

            df_clipped = apply_outlier_clipping(df)

            available_num_cols = [c for c in num_cols if c in df_clipped.columns]

            if len(available_num_cols) > 1:
                # Processar todas as combinações de pares
                df_clean = df_clipped.select(available_num_cols).drop_nulls()

                if len(df_clean) > 0:
                    df_pd = df_clean.to_pandas()

                    for i, col1 in enumerate(available_num_cols):
                        for j, col2 in enumerate(available_num_cols):
                            if i <= j:  # Evitar duplicatas (matriz simétrica)
                                x = df_pd[col1].values.astype(np.float64)  # Float64 para precisão
                                y = df_pd[col2].values.astype(np.float64)

                                n = len(x)
                                acc = corr_acc[col1][col2]
                                acc["n"] += n
                                acc["sum_x"] += np.sum(x)
                                acc["sum_y"] += np.sum(y)
                                acc["sum_xx"] += np.sum(x * x)
                                acc["sum_yy"] += np.sum(y * y)
                                acc["sum_xy"] += np.sum(x * y)

            # Limpeza de memória por arquivo
            del df, df_clipped
            gc.collect()

        except Exception as e:
            print(f"Erro calculando correlação de {file.name}: {e}")
            continue

    # Calcular correlações finais
    corr_matrix = pd.DataFrame(index=num_cols, columns=num_cols, dtype=float)

    for col1 in num_cols:
        for col2 in num_cols:
            if col1 in corr_acc and col2 in corr_acc[col1]:
                acc = corr_acc[col1][col2]
            elif col2 in corr_acc and col1 in corr_acc[col2]:
                acc = corr_acc[col2][col1]
            else:
                corr_matrix.loc[col1, col2] = np.nan
                continue

            if acc["n"] > 1:
                # Fórmula de correlação de Pearson populacional
                n = acc["n"]
                numerator = n * acc["sum_xy"] - acc["sum_x"] * acc["sum_y"]
                denominator = np.sqrt((n * acc["sum_xx"] - acc["sum_x"]**2) * (n * acc["sum_yy"] - acc["sum_y"]**2))

                if denominator > 0:
                    corr = numerator / denominator
                    corr_matrix.loc[col1, col2] = corr
                    corr_matrix.loc[col2, col1] = corr  # Matriz simétrica
                else:
                    corr_matrix.loc[col1, col2] = 0.0
                    corr_matrix.loc[col2, col1] = 0.0
            else:
                corr_matrix.loc[col1, col2] = np.nan
                corr_matrix.loc[col2, col1] = np.nan

    # Diagonal = 1 (correlação de uma variável consigo mesma)
    for col in num_cols:
        corr_matrix.loc[col, col] = 1.0

    # Salvar correlação
    corr_matrix.to_csv(eda_dir / "correlacao_populacional.csv")

    # Plot se houver dados válidos
    if not corr_matrix.isna().all().all():
        try:
            plt.figure(figsize=(15, 12))
            sns.heatmap(corr_matrix.astype(float), cmap="coolwarm", center=0,
                       cbar_kws={'label': 'Correlação de Pearson'})
            plt.title("Matriz de Correlação Populacional (Com Clipping de Outliers)")
            plt.xticks(rotation=45, ha='right')
            plt.yticks(rotation=0)
            plt.tight_layout()
            plt.savefig(eda_dir / "correlacao_populacional.png", dpi=150, bbox_inches='tight')
            plt.close()
        except Exception as e:
            print(f"Erro salvando plot de correlação: {e}")

    # =========================
    # 5) Contagem de linhas por arquivo
    # =========================
    print("\n5) Contando linhas por arquivo...")
    counts = []
    for file in tqdm(PARQUETS, desc="Contagem de linhas"):
        try:
            counts.append({
                "arquivo": file.name,
                "linhas": pa_parquet.read_metadata(file).num_rows
            })
        except Exception as e:
            print(f"Erro contando linhas de {file.name}: {e}")
            continue

    pd.DataFrame(counts).to_csv(eda_dir / "contagem_linhas.csv", index=False)

    # Salvar informações dos percentis utilizados para clipping
    if percentiles_global:
        percentiles_df = []
        for col, info in percentiles_global.items():
            percentiles_df.append({
                "coluna": col,
                "p1_ou_min": info["p1"],
                "p99_ou_max": info["p99"],
                "tipo_clipping": info["type"]
            })
        pd.DataFrame(percentiles_df).to_csv(eda_dir / "clipping_aplicado.csv", index=False)
        print(f"\nInformações de clipping salvas em: {eda_dir}/clipping_aplicado.csv")

    print(f"\nEDA concluída! Resultados salvos em: {eda_dir}")
    print("NOTAS:")
    print("- Coluna 'sst' removida (muitos nulos, não relevante)")
    print("- Outliers extremos (>P99) em wind_shear/intensidade_turbulencia → 0")
    print("- Limites físicos aplicados em t2m, sp, densidade_ar")
    print("- Todas as operações usam Float64 para evitar overflow")
    monitor_memory()

# Execução principal
if __name__ == "__main__":
    print("Iniciando EDA enriquecida em lotes com clipping de outliers...")
    run_eda_enriquecida(n_bins=50)


    print("Processo finalizado!")

Iniciando EDA enriquecida em lotes com clipping de outliers...
Colunas após remoção de sst: ['u10', 'v10', 'u100', 'v100', 't2m', 'sp', 'zust', 'tp', 'number', 'latitude', 'longitude', 'expver', 'altura_torre_m', 'diametro_rotor_m', 'potencia_turbina_mw', 'status_operacao', 'ano_inicio_operacao', 'velocidade_vento_100m', 'velocidade_vento_10m', 'direcao_vento_100m', 'densidade_ar', 'wind_shear', 'intensidade_turbulencia', 'potencia_vento_100m', 'hora_do_dia', 'dia_do_ano', 'mes', 'ano', 'dia', 'potencia_gerada_estimada_mw']
Colunas numéricas: ['u10', 'v10', 'u100', 'v100', 't2m', 'sp', 'zust', 'tp', 'number', 'latitude', 'longitude', 'altura_torre_m', 'diametro_rotor_m', 'potencia_turbina_mw', 'ano_inicio_operacao', 'velocidade_vento_100m', 'velocidade_vento_10m', 'direcao_vento_100m', 'densidade_ar', 'wind_shear', 'intensidade_turbulencia', 'potencia_vento_100m', 'hora_do_dia', 'dia_do_ano', 'mes', 'ano', 'dia', 'potencia_gerada_estimada_mw']
Processando 27 arquivos parquet...
Colunas

Coletando amostras para percentis: 100%|██████████| 27/27 [00:24<00:00,  1.12it/s]

Erro coletando amostras de series_temporais_pivotado.parquet: unable to find column "wind_shear"; valid columns: ["time_", "densidade_ar_01", "densidade_ar_02", "densidade_ar_03", "densidade_ar_04", "densidade_ar_05", "densidade_ar_06", "densidade_ar_07", "densidade_ar_08", "densidade_ar_09", "densidade_ar_1", "densidade_ar_10", "densidade_ar_11", "densidade_ar_12", "densidade_ar_13", "densidade_ar_14", "densidade_ar_15", "densidade_ar_16", "densidade_ar_17", "densidade_ar_18", "densidade_ar_19", "densidade_ar_2", "densidade_ar_20", "densidade_ar_21", "densidade_ar_22", "densidade_ar_23", "densidade_ar_24", "densidade_ar_25", "densidade_ar_26", "densidade_ar_27", "densidade_ar_28", "densidade_ar_3", "densidade_ar_4", "densidade_ar_5", "densidade_ar_6", "densidade_ar_7", "densidade_ar_8", "densidade_ar_9", "densidade_ar_AA-1", "densidade_ar_AA-2", "densidade_ar_AA-3", "densidade_ar_AA-4", "densidade_ar_AA-5", "densidade_ar_AA-6", "densidade_ar_AA-7", "densidade_ar_AA-8", "densidade_ar_A

intensidade_turbulencia: P1=0.017, P99=0.063
t2m: Limites físicos min=200.0, max=330.0
sp: Limites físicos min=20000.0, max=120000.0
densidade_ar: Limites físicos min=0.5, max=1.6

1) Calculando estatísticas mensais (com clipping de outliers)...


Estatísticas mensais: 100%|██████████| 27/27 [00:22<00:00,  1.21it/s]


...


4) Calculando matriz de correlação (populacional) com clipping de outliers...


Calculando correlação populacional: 100%|██████████| 27/27 [01:48<00:00,  4.03s/it]



5) Contando linhas por arquivo...


Contagem de linhas: 100%|██████████| 27/27 [00:01<00:00, 14.43it/s]



Informações de clipping salvas em: C:\Users\Admin\Documents\Puc\IC\Dados\EDA/clipping_aplicado.csv

EDA concluída! Resultados salvos em: C:\Users\Admin\Documents\Puc\IC\Dados\EDA
NOTAS:
- Coluna 'sst' removida (muitos nulos, não relevante)
- Outliers extremos (>P99) em wind_shear/intensidade_turbulencia → 0
- Limites físicos aplicados em t2m, sp, densidade_ar
- Todas as operações usam Float64 para evitar overflow
Uso de memória: 1309.1 MB
Processo finalizado!


In [ ]:
def passada_2_amostragem_percentis(parquet_files, eda_dir, schema_info):
    """PASSADA 2: Amostragem para cálculo de percentis globais - INCLUINDO VELOCIDADES DO VENTO"""
    print("\n" + "="*50)
    print("PASSADA 2: AMOSTRAGEM PARA PERCENTIS GLOBAIS")
    print("="*50)

    num_cols = schema_info['num_cols']

    # Expandir lista para incluir todas as variáveis importantes de vento
    # Incluindo componentes de velocidade do vento e outras variáveis críticas
    wind_related_terms = ['wind_speed', 'velocidade_vento', 'u10', 'v10', 'wind_shear', 'intensidade_turbulencia']
    other_critical_terms = ['power', 'potencia', 'energia', 'temperatura', 't2m']

    # Identificar colunas de vento
    wind_cols = [c for c in num_cols if any(term in c.lower() for term in wind_related_terms)]
    # Identificar outras colunas críticas
    critical_cols = [c for c in num_cols if any(term in c.lower() for term in other_critical_terms)]

    # Combinar todas as colunas para análise de percentis
    percentile_cols = list(set(wind_cols + critical_cols))

    if not percentile_cols:
        print("Nenhuma coluna encontrada para análise de percentis")
        return {}

    print(f"Analisando percentis para {len(percentile_cols)} colunas:")
    print(f"- Velocidade do vento: {wind_cols}")
    print(f"- Outras críticas: {critical_cols}")

    samples_acc = {col: [] for col in percentile_cols}

    for file_path in tqdm(parquet_files, desc="Coletando amostras"):
        try:
            df = pl.read_parquet(file_path)

            if "sst" in df.columns:
                df = df.drop("sst")

            # Calcular velocidade absoluta do vento se componentes estiverem disponíveis
            if 'u10' in df.columns and 'v10' in df.columns:
                df = df.with_columns([
                    (pl.col('u10')**2 + pl.col('v10')**2).sqrt().alias('velocidade_vento_absoluta')
                ])
                # Adicionar à lista se não existir
                if 'velocidade_vento_absoluta' not in samples_acc:
                    samples_acc['velocidade_vento_absoluta'] = []

            # Coletar amostras de cada coluna (cada 15ª linha para ter boa representatividade)
            for col in percentile_cols:
                if col in df.columns:
                    col_sample = df[col][::15].drop_nulls().to_list()
                    samples_acc[col].extend(col_sample)

            # Coletar amostra da velocidade absoluta se foi calculada
            if 'velocidade_vento_absoluta' in df.columns:
                vel_sample = df['velocidade_vento_absoluta'][::15].drop_nulls().to_list()
                samples_acc['velocidade_vento_absoluta'].extend(vel_sample)

            del df
            gc.collect()

        except Exception as e:
            print(f"Erro coletando amostras de {file_path}: {e}")
            continue

    # Calcular percentis para todas as colunas coletadas
    percentis_globais = {}
    percentis_detalhados = []  # Para salvar informações mais detalhadas

    for col in samples_acc.keys():
        if samples_acc[col]:
            values = np.array(samples_acc[col])

            # Calcular múltiplos percentis para análise mais rica
            p1 = np.percentile(values, 1)
            p5 = np.percentile(values, 5)
            p25 = np.percentile(values, 25)
            p50 = np.percentile(values, 50)  # Mediana
            p75 = np.percentile(values, 75)
            p95 = np.percentile(values, 95)
            p99 = np.percentile(values, 99)

            percentis_globais[col] = {"p1": p1, "p99": p99}

            # Informações detalhadas para relatório
            percentis_detalhados.append({
                'coluna': col,
                'p1': p1, 'p5': p5, 'p25': p25, 'p50': p50,
                'p75': p75, 'p95': p95, 'p99': p99,
                'amostra_size': len(values),
                'tipo': 'vento' if any(term in col.lower() for term in wind_related_terms) else 'outras'
            })

            print(f"{col}: P1={p1:.3f}, P50={p50:.3f}, P99={p99:.3f} (n={len(values):,})")
        else:
            percentis_globais[col] = {"p1": 0, "p99": 0}
            percentis_detalhados.append({
                'coluna': col,
                'p1': 0, 'p5': 0, 'p25': 0, 'p50': 0,
                'p75': 0, 'p95': 0, 'p99': 0,
                'amostra_size': 0,
                'tipo': 'vento' if any(term in col.lower() for term in wind_related_terms) else 'outras'
            })

    # Salvar percentis básicos (para compatibilidade com código existente)
    if percentis_globais:
        percentis_df = pd.DataFrame.from_dict(percentis_globais, orient='index')
        percentis_df.to_csv(os.path.join(eda_dir, "percentis_globais.csv"))

    # Salvar análise detalhada de percentis
    if percentis_detalhados:
        percentis_detalhados_df = pd.DataFrame(percentis_detalhados)
        percentis_detalhados_df.to_csv(os.path.join(eda_dir, "analise_percentis_detalhada.csv"), index=False)

        # Criar resumo específico para velocidades do vento
        wind_percentis = percentis_detalhados_df[percentis_detalhados_df['tipo'] == 'vento']
        if not wind_percentis.empty:
            wind_percentis.to_csv(os.path.join(eda_dir, "percentis_velocidades_vento.csv"), index=False)
            print(f"\nResumo das velocidades do vento:")
            for _, row in wind_percentis.iterrows():
                print(f"  {row['coluna']}: Mediana={row['p50']:.2f}, P95={row['p95']:.2f} m/s")

    # Limpar amostras da memória
    del samples_acc
    gc.collect()

    print(f"Uso de memória: {monitor_memory():.1f} MB")
import glob
import polars as pl
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

def monitor_memory():
    """Monitora uso de memória"""
    try:
        import psutil
        process = psutil.Process()
        memory_mb = process.memory_info().rss / 1024 / 1024
        return memory_mb
    except ImportError:
        return 0

def get_schema_info(parquet_files):
    """Extrai informações do schema do primeiro arquivo"""
    if not parquet_files:
        raise ValueError("Nenhum arquivo parquet encontrado")

    print("Analisando schema do primeiro arquivo...")

    first_file = parquet_files[0]
    df_sample = pl.scan_parquet(first_file)

    try:
        schema = df_sample.collect_schema()
    except AttributeError:
        schema = df_sample.schema

    NUMERIC_DTYPES = {pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                      pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                      pl.Float32, pl.Float64}

    time_col = "time"
    high_card_ids = ["CEG", "denominacao_aerogerador", "nome_parque"]

    all_cols = [c for c in schema.keys() if c != "sst"]
    num_cols = [c for c in all_cols if schema[c] in NUMERIC_DTYPES and c != time_col and c not in high_card_ids]
    cat_cols = [c for c in all_cols if schema[c] in {pl.Utf8, pl.Categorical, pl.Boolean} and c not in high_card_ids]
    datetime_cols = [c for c in all_cols if schema[c] in {pl.Datetime, pl.Date, pl.Time, pl.Duration}]
    id_cols = [c for c in high_card_ids if c in schema.keys()]

    if time_col in schema.keys():
        datetime_cols.append(time_col)

    schema_info = {
        'schema': schema,
        'all_cols': all_cols,
        'num_cols': num_cols,
        'cat_cols': cat_cols,
        'datetime_cols': datetime_cols,
        'id_cols': id_cols,
        'time_col': time_col
    }

    print(f"Schema analisado:")
    print(f"- Total colunas: {len(all_cols)}")
    print(f"- Numéricas: {len(num_cols)}")
    print(f"- Categóricas: {len(cat_cols)}")
    print(f"- Datetime: {len(datetime_cols)}")
    print(f"- IDs alta cardinalidade: {len(id_cols)}")

    return schema_info

def passada_1_estatisticas_basicas(parquet_files, eda_dir, schema_info):
    """PASSADA 1: Estatísticas básicas por arquivo"""
    print("\n" + "="*50)
    print("PASSADA 1: ESTATÍSTICAS BÁSICAS POR ARQUIVO")
    print("="*50)

    num_cols = schema_info['num_cols']
    parciais_dir = os.path.join(eda_dir, "parciais", "estatisticas")
    os.makedirs(parciais_dir, exist_ok=True)

    for file_path in tqdm(parquet_files, desc="Processando estatísticas básicas"):
        try:
            df = pl.read_parquet(file_path)
            file_name = os.path.basename(file_path)

            if "sst" in df.columns:
                df = df.drop("sst")

            stats_rows = []

            for col in num_cols:
                if col in df.columns:
                    col_data = df[col].drop_nulls()

                    if len(col_data) > 0:
                        count = len(col_data)
                        sum_val = float(col_data.sum())
                        sum_sq = float((col_data.cast(pl.Float64) ** 2).sum())
                        min_val = float(col_data.min())
                        max_val = float(col_data.max())

                        stats_rows.append({
                            'arquivo': file_name,
                            'coluna': col,
                            'count': count,
                            'sum': sum_val,
                            'sum_sq': sum_sq,
                            'min': min_val,
                            'max': max_val
                        })

            if stats_rows:
                stats_df = pd.DataFrame(stats_rows)
                arquivo_stats = os.path.join(parciais_dir, f"stats_{os.path.splitext(file_name)[0]}.csv")
                stats_df.to_csv(arquivo_stats, index=False)

            del df, col_data, stats_df
            gc.collect()

        except Exception as e:
            print(f"Erro processando {file_path}: {e}")
            continue

    print(f"Estatísticas básicas salvas em: {parciais_dir}")
    print(f"Uso de memória: {monitor_memory():.1f} MB")

def passada_2_amostragem_percentis(parquet_files, eda_dir, schema_info):
    """PASSADA 2: Amostragem para cálculo de percentis globais"""
    print("\n" + "="*50)
    print("PASSADA 2: AMOSTRAGEM PARA PERCENTIS GLOBAIS")
    print("="*50)

    num_cols = schema_info['num_cols']
    percentile_cols = [c for c in ["wind_shear", "intensidade_turbulencia"] if c in num_cols]

    if not percentile_cols:
        print("Nenhuma coluna necessita de clipping por percentis")
        return {}

    samples_acc = {col: [] for col in percentile_cols}

    for file_path in tqdm(parquet_files, desc="Coletando amostras"):
        try:
            df = pl.read_parquet(file_path)

            if "sst" in df.columns:
                df = df.drop("sst")

            for col in percentile_cols:
                if col in df.columns:
                    col_sample = df[col][::20].drop_nulls().to_list()
                    samples_acc[col].extend(col_sample)

            del df
            gc.collect()

        except Exception as e:
            print(f"Erro coletando amostras de {file_path}: {e}")
            continue

    percentis_globais = {}
    for col in percentile_cols:
        if samples_acc[col]:
            values = np.array(samples_acc[col])
            p1 = np.percentile(values, 1)
            p99 = np.percentile(values, 99)
            percentis_globais[col] = {"p1": p1, "p99": p99}
            print(f"{col}: P1={p1:.3f}, P99={p99:.3f}")
        else:
            percentis_globais[col] = {"p1": 0, "p99": 0}

    if percentis_globais:
        percentis_df = pd.DataFrame.from_dict(percentis_globais, orient='index')
        percentis_df.to_csv(os.path.join(eda_dir, "percentis_globais.csv"))

    del samples_acc
    gc.collect()

    print(f"Uso de memória: {monitor_memory():.1f} MB")
    return percentis_globais

def passada_3_histogramas(parquet_files, eda_dir, schema_info, n_bins=50):
    """PASSADA 3: Cálculo de histogramas por arquivo"""
    print("\n" + "="*50)
    print("PASSADA 3: HISTOGRAMAS POR ARQUIVO")
    print("="*50)

    num_cols = schema_info['num_cols']
    parciais_dir = os.path.join(eda_dir, "parciais", "histogramas")
    os.makedirs(parciais_dir, exist_ok=True)

    stats_dir = os.path.join(eda_dir, "parciais", "estatisticas")
    stats_files = glob.glob(os.path.join(stats_dir, "*.csv"))

    if not stats_files:
        print("Estatísticas básicas não encontradas. Execute passada_1 primeiro.")
        return

    global_limits = {}
    for stats_file in stats_files:
        stats_df = pd.read_csv(stats_file)
        for _, row in stats_df.iterrows():
            col = row['coluna']
            if col not in global_limits:
                global_limits[col] = {'min': row['min'], 'max': row['max']}
            else:
                global_limits[col]['min'] = min(global_limits[col]['min'], row['min'])
                global_limits[col]['max'] = max(global_limits[col]['max'], row['max'])

    for file_path in tqdm(parquet_files, desc="Calculando histogramas"):
        try:
            df = pl.read_parquet(file_path)
            file_name = os.path.basename(file_path)

            if "sst" in df.columns:
                df = df.drop("sst")

            hist_rows = []

            for col in num_cols:
                if col in df.columns and col in global_limits:
                    col_data = df[col].drop_nulls()

                    if len(col_data) == 0:
                        continue

                    cmin = global_limits[col]['min']
                    cmax = global_limits[col]['max']

                    if cmin == cmax:
                        hist_rows.append({
                            'arquivo': file_name,
                            'coluna': col,
                            'bin': 0,
                            'bin_start': cmin,
                            'bin_end': cmax,
                            'count': len(col_data)
                        })
                        continue

                    values = col_data.to_numpy()
                    hist, edges = np.histogram(values, bins=n_bins, range=(cmin, cmax))

                    for i in range(n_bins):
                        hist_rows.append({
                            'arquivo': file_name,
                            'coluna': col,
                            'bin': i,
                            'bin_start': edges[i],
                            'bin_end': edges[i+1],
                            'count': int(hist[i])
                        })

            if hist_rows:
                hist_df = pd.DataFrame(hist_rows)
                arquivo_hist = os.path.join(parciais_dir, f"hist_{os.path.splitext(file_name)[0]}.csv")
                hist_df.to_csv(arquivo_hist, index=False)

            del df, hist_df
            gc.collect()

        except Exception as e:
            print(f"Erro calculando histograma de {file_path}: {e}")
            continue

    print(f"Histogramas salvos em: {parciais_dir}")
    print(f"Uso de memória: {monitor_memory():.1f} MB")

def passada_4_validacao_regras(parquet_files, eda_dir, schema_info):
    """PASSADA 4: Validação de regras de negócio por arquivo"""
    print("\n" + "="*50)
    print("PASSADA 4: VALIDAÇÃO DE REGRAS DE NEGÓCIO")
    print("="*50)

    parciais_dir = os.path.join(eda_dir, "parciais", "validacao")
    os.makedirs(parciais_dir, exist_ok=True)

    def get_validation_rules(df_columns):
        rules = []

        # 1. Regras de velocidade do vento - APENAS MAGNITUDES, NÃO COMPONENTES
        wind_speed_cols = [c for c in df_columns if any(term in c.lower() for term in ['wind_speed', 'velocidade_vento'])
                          and not any(comp in c.lower() for comp in ['u10', 'v10', 'u100', 'v100'])]
        for col in wind_speed_cols:
            rules.extend([
                (f"{col}_velocidade_negativa", pl.col(col) < 0),  # Apenas magnitudes não podem ser negativas
                (f"{col}_velocidade_muito_alta", pl.col(col) > 60),
                (f"{col}_velocidade_zero_excessiva", pl.col(col) == 0),
            ])

        # 2. Regras de direção do vento
        wind_dir_cols = [c for c in df_columns if any(term in c.lower() for term in ['wind_direction', 'direcao_vento', 'dir'])]
        for col in wind_dir_cols:
            rules.extend([
                (f"{col}_direcao_invalida", (pl.col(col) < 0) | (pl.col(col) > 360)),
            ])

        # 3. Regras de potência
        power_cols = [c for c in df_columns if any(term in c.lower() for term in ['power', 'potencia', 'energia'])]
        for col in power_cols:
            rules.extend([
                (f"{col}_potencia_negativa", pl.col(col) < 0),
                (f"{col}_potencia_muito_alta", pl.col(col) > 10000),
            ])

        # 4. Regras de temperatura - CORRIGIDAS PARA KELVIN (ERA5)
        temp_cols = [c for c in df_columns if any(term in c.lower() for term in ['temp', 't2m', 'temperatura'])]
        for col in temp_cols:
            rules.extend([
                (f"{col}_temperatura_kelvin_muito_baixa", pl.col(col) < 200),  # ~-73°C
                (f"{col}_temperatura_kelvin_muito_alta", pl.col(col) > 330),   # ~57°C
                (f"{col}_temperatura_kelvin_impossivel", pl.col(col) < 100),   # Fisicamente impossível
            ])

        # 5. Regras de pressão
        pressure_cols = [c for c in df_columns if any(term in c.lower() for term in ['pressure', 'pressao', 'sp'])]
        for col in pressure_cols:
            rules.extend([
                (f"{col}_pressao_muito_baixa", pl.col(col) < 50000),
                (f"{col}_pressao_muito_alta", pl.col(col) > 110000),
            ])

        # 6. Regras de densidade do ar
        density_cols = [c for c in df_columns if any(term in c.lower() for term in ['densidade', 'density'])]
        for col in density_cols:
            rules.extend([
                (f"{col}_densidade_muito_baixa", pl.col(col) < 0.8),
                (f"{col}_densidade_muito_alta", pl.col(col) > 1.4),
                (f"{col}_densidade_negativa", pl.col(col) < 0),
            ])

        # 7. Regras gerais de qualidade
        for col in schema_info['num_cols']:
            if col in df_columns:
                rules.extend([
                    (f"{col}_valores_inf", pl.col(col).is_infinite()),
                    (f"{col}_valores_nan", pl.col(col).is_nan()),
                ])

        return rules

    for file_path in tqdm(parquet_files, desc="Validando regras"):
        try:
            df = pl.read_parquet(file_path)
            file_name = os.path.basename(file_path)

            if "sst" in df.columns:
                df = df.drop("sst")

            rules = get_validation_rules(df.columns)
            validation_rows = []

            for rule_name, condition in rules:
                try:
                    violations = df.select(condition.sum().alias("violations")).item()

                    if violations > 0:
                        validation_rows.append({
                            'arquivo': file_name,
                            'regra': rule_name,
                            'violacoes': violations,
                            'coluna': rule_name.split('_')[0] if '_' in rule_name else rule_name
                        })

                except Exception:
                    continue

            if validation_rows:
              validation_df = pd.DataFrame(validation_rows)
              arquivo_val = os.path.join(parciais_dir, f"val_{os.path.splitext(file_name)[0]}.csv")
              validation_df.to_csv(arquivo_val, index=False)
              del validation_df  # Só deleta se foi criado

            del df  # Sempre deleta df
            gc.collect()

        except Exception as e:
            print(f"Erro validando {file_path}: {e}")
            continue

    print(f"Validações salvas em: {parciais_dir}")
    print(f"Uso de memória: {monitor_memory():.1f} MB")

def passada_5_analise_temporal(parquet_files, eda_dir, schema_info):
    """PASSADA 5: Análise temporal/sazonal por arquivo"""
    print("\n" + "="*50)
    print("PASSADA 5: ANÁLISE TEMPORAL/SAZONAL")
    print("="*50)

    num_cols = schema_info['num_cols']
    time_col = schema_info['time_col']

    parciais_dir = os.path.join(eda_dir, "parciais", "temporal")
    os.makedirs(parciais_dir, exist_ok=True)

    for file_path in tqdm(parquet_files, desc="Analisando sazonalidade"):
        try:
            df = pl.read_parquet(file_path)
            file_name = os.path.basename(file_path)

            if "sst" in df.columns:
                df = df.drop("sst")

            if time_col not in df.columns:
                continue

            df_temporal = df.with_columns([
                pl.col(time_col).cast(pl.Datetime).alias("datetime"),
                pl.col(time_col).cast(pl.Datetime).dt.month().alias("mes"),
                pl.col(time_col).cast(pl.Datetime).dt.hour().alias("hora"),
                pl.col(time_col).cast(pl.Datetime).dt.weekday().alias("dia_semana")
            ])

            temporal_rows = []

            for periodo, col_periodo in [("mes", "mes"), ("hora", "hora"), ("dia_semana", "dia_semana")]:
                for col in num_cols:
                    if col in df_temporal.columns:
                        try:
                            agg_data = (df_temporal
                                      .select([col_periodo, col])
                                      .drop_nulls()
                                      .group_by(col_periodo)
                                      .agg([
                                          pl.col(col).count().alias("count"),
                                          pl.col(col).sum().alias("sum")
                                      ]))

                            for row in agg_data.to_dicts():
                                temporal_rows.append({
                                    'arquivo': file_name,
                                    'periodo_tipo': periodo,
                                    'periodo_valor': row[col_periodo],
                                    'coluna': col,
                                    'count': row['count'],
                                    'sum': row['sum']
                                })

                        except Exception:
                            continue

            if temporal_rows:
                temporal_df = pd.DataFrame(temporal_rows)
                arquivo_temp = os.path.join(parciais_dir, f"temp_{os.path.splitext(file_name)[0]}.csv")
                temporal_df.to_csv(arquivo_temp, index=False)

            del df, df_temporal, temporal_df
            gc.collect()

        except Exception as e:
            print(f"Erro na análise temporal de {file_path}: {e}")
            continue

    print(f"Análise temporal salva em: {parciais_dir}")
    print(f"Uso de memória: {monitor_memory():.1f} MB")

def consolidar_resultados(eda_dir):
    """Consolida todos os resultados parciais em arquivos finais"""
    print("\n" + "="*50)
    print("CONSOLIDANDO RESULTADOS FINAIS")
    print("="*50)

    consolidados_dir = os.path.join(eda_dir, "consolidados")
    os.makedirs(consolidados_dir, exist_ok=True)

    # 1. Consolidar estatísticas básicas
    print("1. Consolidando estatísticas básicas...")
    stats_files = glob.glob(os.path.join(eda_dir, "parciais", "estatisticas", "*.csv"))
    if stats_files:
        all_stats = []
        for file in stats_files:
            df = pd.read_csv(file)
            all_stats.append(df)

        combined_stats = pd.concat(all_stats, ignore_index=True)

        global_stats = combined_stats.groupby('coluna').agg({
            'count': 'sum',
            'sum': 'sum',
            'sum_sq': 'sum',
            'min': 'min',
            'max': 'max'
        }).reset_index()

        global_stats['mean'] = global_stats['sum'] / global_stats['count']
        global_stats['variance'] = (global_stats['sum_sq'] / global_stats['count']) - (global_stats['mean'] ** 2)
        global_stats['std'] = np.sqrt(np.maximum(global_stats['variance'], 0))

        global_stats.to_csv(os.path.join(consolidados_dir, "estatisticas_globais.csv"), index=False)
        combined_stats.to_csv(os.path.join(consolidados_dir, "estatisticas_por_arquivo.csv"), index=False)

    # 2. Consolidar histogramas
    print("2. Consolidando histogramas...")
    hist_files = glob.glob(os.path.join(eda_dir, "parciais", "histogramas", "*.csv"))
    if hist_files:
        all_hists = []
        for file in hist_files:
            df = pd.read_csv(file)
            all_hists.append(df)

        combined_hists = pd.concat(all_hists, ignore_index=True)
        global_hists = combined_hists.groupby(['coluna', 'bin', 'bin_start', 'bin_end'])['count'].sum().reset_index()
        global_hists.to_csv(os.path.join(consolidados_dir, "histogramas_globais.csv"), index=False)

    # 3. Consolidar validações
    print("3. Consolidando validações...")
    val_files = glob.glob(os.path.join(eda_dir, "parciais", "validacao", "*.csv"))
    if val_files:
        all_validations = []
        for file in val_files:
            df = pd.read_csv(file)
            all_validations.append(df)

        combined_validations = pd.concat(all_validations, ignore_index=True)

        summary_validations = combined_validations.groupby(['regra', 'coluna'])['violacoes'].sum().reset_index()
        summary_validations['criticidade'] = summary_validations['violacoes'].apply(
            lambda x: 'alta' if x > 1000 else 'media' if x > 100 else 'baixa'
        )

        combined_validations.to_csv(os.path.join(consolidados_dir, "validacoes_detalhadas.csv"), index=False)
        summary_validations.to_csv(os.path.join(consolidados_dir, "validacoes_resumo.csv"), index=False)

    # 4. Consolidar análise temporal
    print("4. Consolidando análise temporal...")
    temp_files = glob.glob(os.path.join(eda_dir, "parciais", "temporal", "*.csv"))
    if temp_files:
        all_temporal = []
        for file in temp_files:
            df = pd.read_csv(file)
            all_temporal.append(df)

        combined_temporal = pd.concat(all_temporal, ignore_index=True)

        temporal_agg = combined_temporal.groupby(['periodo_tipo', 'periodo_valor', 'coluna']).agg({
            'count': 'sum',
            'sum': 'sum'
        }).reset_index()
        temporal_agg['mean'] = temporal_agg['sum'] / temporal_agg['count']

        temporal_agg.to_csv(os.path.join(consolidados_dir, "analise_temporal_consolidada.csv"), index=False)

    print("Consolidação concluída!")

def gerar_visualizacoes_finais(eda_dir):
    """Gera visualizações a partir dos dados consolidados"""
    print("\n" + "="*50)
    print("GERANDO VISUALIZAÇÕES FINAIS")
    print("="*50)

    consolidados_dir = os.path.join(eda_dir, "consolidados")
    viz_dir = os.path.join(eda_dir, "visualizacoes")
    os.makedirs(viz_dir, exist_ok=True)

    try:
        # 1. Dashboard de estatísticas globais
        stats_path = os.path.join(consolidados_dir, "estatisticas_globais.csv")
        if os.path.exists(stats_path):
            stats_df = pd.read_csv(stats_path)

            important_vars = ['wind_speed', 'power', 'temperature', 't2m', 'densidade_ar', 'pressure']
            available_vars = [v for v in important_vars if v in stats_df['coluna'].values][:6]

            if available_vars:
                dashboard_data = stats_df[stats_df['coluna'].isin(available_vars)]

                fig, axes = plt.subplots(2, 3, figsize=(18, 10))
                axes = axes.flatten()

                metrics = ['mean', 'std', 'min', 'max', 'count']
                metric_names = ['Média', 'Desvio Padrão', 'Mínimo', 'Máximo', 'Contagem']

                for i, (metric, name) in enumerate(zip(metrics, metric_names)):
                    if i < len(axes) and metric in dashboard_data.columns:
                        values = dashboard_data[metric].values
                        labels = dashboard_data['coluna'].values

                        bars = axes[i].bar(range(len(values)), values, alpha=0.7)
                        axes[i].set_title(name)
                        axes[i].set_xticks(range(len(labels)))
                        axes[i].set_xticklabels(labels, rotation=45, ha='right')

                        for bar, value in zip(bars, values):
                            height = bar.get_height()
                            axes[i].text(bar.get_x() + bar.get_width()/2., height,
                                       f'{value:.1e}' if abs(value) >= 1000 else f'{value:.2f}',
                                       ha='center', va='bottom', fontsize=8)

                if len(axes) > 5:
                    axes[5].text(0.5, 0.5, 'Espaço Reservado',
                                transform=axes[5].transAxes, ha='center', va='center')
                    axes[5].set_title('Reservado')

                plt.tight_layout()
                plt.savefig(os.path.join(viz_dir, "dashboard_estatisticas_globais.png"), dpi=200, bbox_inches='tight')
                plt.close()

        # 2. Violações de regras
        val_path = os.path.join(consolidados_dir, "validacoes_resumo.csv")
        if os.path.exists(val_path):
            val_df = pd.read_csv(val_path)

            if not val_df.empty:
                top_violations = val_df.nlargest(15, 'violacoes')

                plt.figure(figsize=(14, 8))

                colors_map = {'alta': 'red', 'media': 'orange', 'baixa': 'yellow'}
                bar_colors = [colors_map.get(crit, 'gray') for crit in top_violations['criticidade']]

                bars = plt.barh(range(len(top_violations)), top_violations['violacoes'], color=bar_colors)

                labels = [f"{rule[:40]}..." if len(rule) > 40 else rule
                         for rule in top_violations['regra']]
                plt.yticks(range(len(top_violations)), labels)

                plt.xlabel('Número de Violações')
                plt.title('Top 15 Violações de Regras de Negócio')

                for i, (bar, value) in enumerate(zip(bars, top_violations['violacoes'])):
                    plt.text(value + max(top_violations['violacoes']) * 0.01, i,
                            f'{int(value):,}', va='center', fontsize=9)

                from matplotlib.patches import Patch
                legend_elements = [
                    Patch(facecolor='red', label='Alta (>1000)'),
                    Patch(facecolor='orange', label='Média (100-1000)'),
                    Patch(facecolor='yellow', label='Baixa (<100)')
                ]
                plt.legend(handles=legend_elements, title='Criticidade', loc='lower right')

                plt.tight_layout()
                plt.savefig(os.path.join(viz_dir, "top_violacoes_regras.png"), dpi=200, bbox_inches='tight')
                plt.close()

        # 3. Distribuições
        hist_path = os.path.join(consolidados_dir, "histogramas_globais.csv")
        if os.path.exists(hist_path):
            hist_df = pd.read_csv(hist_path)

            if not hist_df.empty:
                important_dist = ['wind_speed', 'power', 'temperature', 'densidade_ar']
                available_dist = [v for v in important_dist if v in hist_df['coluna'].values][:4]

                if available_dist:
                    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
                    axes = axes.flatten()

                    for i, var in enumerate(available_dist):
                        if i < len(axes):
                            var_hist = hist_df[hist_df['coluna'] == var].sort_values('bin')

                            if len(var_hist) > 0:
                                bin_centers = (var_hist['bin_start'] + var_hist['bin_end']) / 2
                                bin_width = (var_hist['bin_end'] - var_hist['bin_start']).iloc[0]

                                axes[i].bar(bin_centers, var_hist['count'],
                                           width=bin_width * 0.8, alpha=0.7,
                                           edgecolor='black', linewidth=0.5)

                                axes[i].set_title(f'Distribuição - {var}')
                                axes[i].set_xlabel('Valor')
                                axes[i].set_ylabel('Frequência')
                                axes[i].grid(True, alpha=0.3, axis='y')

                    for i in range(len(available_dist), len(axes)):
                        fig.delaxes(axes[i])

                    plt.tight_layout()
                    plt.savefig(os.path.join(viz_dir, "distribuicoes_principais.png"), dpi=200, bbox_inches='tight')
                    plt.close()

        print("Visualizações geradas com sucesso!")
        print(f"Arquivos salvos em: {viz_dir}")

    except Exception as e:
        print(f"Erro gerando visualizações: {e}")
        import traceback
        traceback.print_exc()

def gerar_relatorio_final(eda_dir):
    """Gera um relatório final em markdown com os principais achados"""
    print("\n" + "="*50)
    print("GERANDO RELATÓRIO FINAL")
    print("="*50)

    consolidados_dir = os.path.join(eda_dir, "consolidados")
    relatorio_path = os.path.join(eda_dir, "RELATORIO_EDA_FINAL.md")

    with open(relatorio_path, 'w', encoding='utf-8') as f:
        f.write("# Relatório de EDA - Dados de Energia Eólica\n\n")
        f.write(f"**Data de geração:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

        f.write("## Resumo Executivo\n\n")

        stats_path = os.path.join(consolidados_dir, "estatisticas_globais.csv")
        if os.path.exists(stats_path):
            stats_df = pd.read_csv(stats_path)
            f.write(f"- **Variáveis numéricas analisadas:** {len(stats_df)}\n")

            total_records = stats_df['count'].max() if not stats_df.empty else 0
            f.write(f"- **Total de registros processados:** {total_records:,}\n")

        val_path = os.path.join(consolidados_dir, "validacoes_resumo.csv")
        if os.path.exists(val_path):
            val_df = pd.read_csv(val_path)
            if not val_df.empty:
                total_violations = val_df['violacoes'].sum()
                high_crit = len(val_df[val_df['criticidade'] == 'alta'])
                f.write(f"- **Total de violações encontradas:** {total_violations:,}\n")
                f.write(f"- **Violações de alta criticidade:** {high_crit}\n")

        f.write("\n## Principais Achados\n\n")

        f.write("### Qualidade dos Dados\n")
        if os.path.exists(val_path):
            val_df = pd.read_csv(val_path)
            if not val_df.empty:
                top_violations = val_df.nlargest(5, 'violacoes')
                f.write("**Top 5 problemas de qualidade:**\n")
                for _, row in top_violations.iterrows():
                    f.write(f"- {row['regra']}: {row['violacoes']:,} violações ({row['criticidade']} criticidade)\n")
                f.write("\n")

        f.write("### Estatísticas das Principais Variáveis\n")
        if os.path.exists(stats_path):
            stats_df = pd.read_csv(stats_path)
            important_vars = ['wind_speed', 'power', 'temperature', 't2m', 'densidade_ar']

            for var in important_vars:
                var_stats = stats_df[stats_df['coluna'] == var]
                if not var_stats.empty:
                    row = var_stats.iloc[0]
                    f.write(f"**{var}:**\n")
                    f.write(f"- Média: {row['mean']:.3f}\n")
                    f.write(f"- Desvio padrão: {row['std']:.3f}\n")
                    f.write(f"- Mín/Máx: {row['min']:.3f} / {row['max']:.3f}\n")
                    f.write(f"- Registros válidos: {row['count']:,}\n\n")

        f.write("## Arquivos Processados\n\n")
        stats_arquivo_path = os.path.join(consolidados_dir, "estatisticas_por_arquivo.csv")
        if os.path.exists(stats_arquivo_path):
            stats_files = pd.read_csv(stats_arquivo_path)
            unique_files = stats_files['arquivo'].nunique()
            f.write(f"**Total de arquivos processados:** {unique_files}\n")

            max_records = stats_files.groupby('arquivo')['count'].sum()
            top_file = max_records.idxmax()
            max_count = max_records.max()
            f.write(f"**Arquivo com mais registros:** {top_file} ({max_count:,} registros)\n\n")

        f.write("## Recomendações\n\n")
        f.write("### Limpeza de Dados\n")
        f.write("1. **Investigar violações de alta criticidade** - Focar nas regras com mais de 1.000 violações\n")
        f.write("2. **Validar outliers** - Especialmente em variáveis de velocidade do vento e potência\n")
        f.write("3. **Verificar consistência temporal** - Alguns arquivos podem ter problemas de timestamp\n\n")

        f.write("### Análises Futuras\n")
        f.write("1. **Modelagem preditiva** - Dados estão limpos o suficiente para ML\n")
        f.write("2. **Análise de correlação espacial** - Usar coordenadas geográficas\n")
        f.write("3. **Detecção de anomalias** - Implementar algoritmos de detecção automática\n\n")

        f.write("## Arquivos Gerados\n\n")
        f.write("### Dados Consolidados\n")
        consolidados_files = glob.glob(os.path.join(consolidados_dir, "*.csv"))
        for arquivo in consolidados_files:
            f.write(f"- `{os.path.basename(arquivo)}`\n")

        f.write("\n### Visualizações\n")
        viz_dir = os.path.join(eda_dir, "visualizacoes")
        viz_files = glob.glob(os.path.join(viz_dir, "*.png"))
        for arquivo in viz_files:
            f.write(f"- `{os.path.basename(arquivo)}`\n")

        f.write(f"\n---\n*Relatório gerado automaticamente pelo sistema de EDA*")

    print(f"Relatório final salvo em: {relatorio_path}")

# FUNÇÃO PRINCIPAL PARA EXECUTAR TUDO
def executar_eda_completa():
    """Executa toda a EDA usando as variáveis já definidas no ambiente"""
    print("INICIANDO EDA MULTI-PASSADAS COM CONTROLE DE MEMÓRIA")
    print("="*60)

    # Usar as variáveis já definidas no ambiente
    parquet_files = PARQUETS  # Sua variável PARQUETS já definida
    eda_output_dir = EDA_DIR  # Sua variável EDA_DIR já definida

    if not parquet_files:
        print(f"ERRO: Nenhum arquivo encontrado na variável PARQUETS")
        return

    print(f"Encontrados {len(parquet_files)} arquivos parquet para processar")
    print(f"Diretório de saída: {eda_output_dir}")

    # Criar subdiretórios para resultados parciais
    os.makedirs(os.path.join(eda_output_dir, "parciais"), exist_ok=True)
    os.makedirs(os.path.join(eda_output_dir, "consolidados"), exist_ok=True)
    os.makedirs(os.path.join(eda_output_dir, "visualizacoes"), exist_ok=True)

    # Obter informações do schema
    schema_info = get_schema_info(parquet_files)

    # Salvar informações do schema
    with open(os.path.join(eda_output_dir, "schema_info.txt"), "w") as f:
        for key, value in schema_info.items():
            if key != 'schema':
                f.write(f"{key}: {value}\n")

    # Executar passadas
    try:
        print(f"Memória inicial: {monitor_memory():.1f} MB")

        passada_1_estatisticas_basicas(parquet_files, eda_output_dir, schema_info)
        percentis = passada_2_amostragem_percentis(parquet_files, eda_output_dir, schema_info)
        passada_3_histogramas(parquet_files, eda_output_dir, schema_info)
        passada_4_validacao_regras(parquet_files, eda_output_dir, schema_info)
        passada_5_analise_temporal(parquet_files, eda_output_dir, schema_info)

        # Consolidar resultados
        consolidar_resultados(eda_output_dir)

        # Gerar visualizações
        gerar_visualizacoes_finais(eda_output_dir)

        # Gerar relatório final
        gerar_relatorio_final(eda_output_dir)

        print("\n" + "="*60)
        print("EDA MULTI-PASSADAS CONCLUÍDA COM SUCESSO!")
        print("="*60)
        print(f"Resultados salvos em: {eda_output_dir}")
        print(f"Uso final de memória: {monitor_memory():.1f} MB")

    except Exception as e:
        print(f"Erro durante a execução: {e}")
        import traceback
        traceback.print_exc()

# EXECUTAR A EDA
executar_eda_completa()

INICIANDO EDA MULTI-PASSADAS COM CONTROLE DE MEMÓRIA
Encontrados 27 arquivos parquet para processar
Diretório de saída: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA
Analisando schema do primeiro arquivo...
Schema analisado:
- Total colunas: 34
- Numéricas: 28
- Categóricas: 2
- Datetime: 1
- IDs alta cardinalidade: 3
Memória inicial: 338.8 MB

PASSADA 1: ESTATÍSTICAS BÁSICAS POR ARQUIVO


Processando estatísticas básicas: 100%|██████████| 27/27 [00:06<00:00,  4.00it/s]


Erro processando C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados//Series_Temporais\series_temporais_pivotado.parquet: cannot access local variable 'col_data' where it is not associated with a value
Estatísticas básicas salvas em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\parciais\estatisticas
Uso de memória: 1158.0 MB

PASSADA 2: AMOSTRAGEM PARA PERCENTIS GLOBAIS


Coletando amostras: 100%|██████████| 27/27 [00:06<00:00,  4.42it/s]


wind_shear: P1=0.070, P99=0.369
intensidade_turbulencia: P1=0.019, P99=0.068
Uso de memória: 1217.1 MB

PASSADA 3: HISTOGRAMAS POR ARQUIVO


...

Erro calculando histograma de C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados//Series_Temporais\series_temporais_pivotado.parquet: cannot access local variable 'hist_df' where it is not associated with a value
Histogramas salvos em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\parciais\histogramas
Uso de memória: 1253.5 MB

PASSADA 4: VALIDAÇÃO DE REGRAS DE NEGÓCIO


Validando regras: 100%|██████████| 27/27 [04:21<00:00,  9.70s/it]


Validações salvas em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\parciais\validacao
Uso de memória: 1286.0 MB

PASSADA 5: ANÁLISE TEMPORAL/SAZONAL


Analisando sazonalidade: 100%|██████████| 27/27 [00:22<00:00,  1.19it/s]


Análise temporal salva em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\parciais\temporal
Uso de memória: 1275.3 MB

CONSOLIDANDO RESULTADOS FINAIS
1. Consolidando estatísticas básicas...
2. Consolidando histogramas...
3. Consolidando validações...
4. Consolidando análise temporal...
Consolidação concluída!

GERANDO VISUALIZAÇÕES FINAIS
Visualizações geradas com sucesso!
Arquivos salvos em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\visualizacoes

GERANDO RELATÓRIO FINAL
Relatório final salvo em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA\RELATORIO_EDA_FINAL.md

EDA MULTI-PASSADAS CONCLUÍDA COM SUCESSO!
Resultados salvos em: C:\\Users\\Admin\\Documents\\Puc\\IC\\Dados\EDA
Uso final de memória: 1302.9 MB


In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob

def inspecionar_percentis_globais():
    """Analisa o arquivo de percentis globais"""
    print("="*60)
    print("1. PERCENTIS GLOBAIS")
    print("="*60)

    if os.path.exists(percent_glob):
        percentis_df = pd.read_csv(percent_glob)
        print("Percentis calculados:")
        print(percentis_df)
        print(f"\nVariáveis com percentis: {list(percentis_df.index)}")

        # Verificar se velocidades de vento foram calculadas
        wind_vars = ['velocidade_vento_10m', 'velocidade_vento_100m', 'wind_speed']
        wind_found = [var for var in wind_vars if var in percentis_df.index]
        print(f"Variáveis de velocidade de vento encontradas: {wind_found}")

        if not wind_found:
            print("⚠️  ATENÇÃO: Percentis de velocidade de vento NÃO foram calculados!")

        return percentis_df
    else:
        print(f"❌ Arquivo não encontrado: {percent_glob}")
        return None

def inspecionar_estatisticas_globais():
    """Analisa estatísticas globais para definir clipping alternativo"""
    print("\n" + "="*60)
    print("2. ESTATÍSTICAS GLOBAIS")
    print("="*60)

    if os.path.exists(global_stat):
        stats_df = pd.read_csv(global_stat)

        # Focar nas variáveis que precisam de clipping
        vars_interesse = [
            'wind_shear', 'intensidade_turbulencia',
            'velocidade_vento_10m', 'velocidade_vento_100m',
            'potencia_gerada_estimada_mw',
            't2m'  # Para verificar se está em Kelvin
        ]

        print("Estatísticas das variáveis de interesse:")
        print("-" * 80)

        for var in vars_interesse:
            var_stats = stats_df[stats_df['coluna'] == var]
            if not var_stats.empty:
                row = var_stats.iloc[0]
                print(f"\n{var}:")
                print(f"  Count: {row['count']:,}")
                print(f"  Min/Max: {row['min']:.3f} / {row['max']:.3f}")
                print(f"  Média ± Std: {row['mean']:.3f} ± {row['std']:.3f}")

                # Calcular percentis aproximados usando média ± N*std
                p1_approx = row['mean'] - 2.33 * row['std']  # aproximadamente P1
                p99_approx = row['mean'] + 2.33 * row['std']  # aproximadamente P99
                print(f"  P1/P99 aproximados: {p1_approx:.3f} / {p99_approx:.3f}")

                # Verificações específicas
                if 't2m' in var and row['mean'] > 100:
                    print("  ✓ Temperatura em Kelvin (média > 100)")
                elif 't2m' in var:
                    print("  ⚠️  Temperatura pode estar em Celsius")

            else:
                print(f"\n{var}: ❌ NÃO ENCONTRADA")

        return stats_df
    else:
        print(f"❌ Arquivo não encontrado: {global_stat}")
        return None

def inspecionar_validacoes():
    """Analisa violações para entender problemas reais"""
    print("\n" + "="*60)
    print("3. VIOLAÇÕES DE VALIDAÇÃO")
    print("="*60)

    if os.path.exists(valid):
        val_df = pd.read_csv(valid)

        print(f"Total de tipos de violações: {len(val_df)}")
        print(f"Total de violações: {val_df['violacoes'].sum():,}")

        # Top violações
        print("\nTop 10 violações mais frequentes:")
        top_violations = val_df.nlargest(10, 'violacoes')
        for _, row in top_violations.iterrows():
            print(f"  {row['regra']}: {row['violacoes']:,} violações")

        # Analisar violações relacionadas ao vento
        print("\nViolações relacionadas ao vento:")
        wind_violations = val_df[val_df['regra'].str.contains('wind|vento|u10|v10|u100|v100', case=False)]
        for _, row in wind_violations.iterrows():
            print(f"  {row['regra']}: {row['violacoes']:,}")

        # Analisar violações de temperatura
        print("\nViolações relacionadas à temperatura:")
        temp_violations = val_df[val_df['regra'].str.contains('temp|t2m', case=False)]
        for _, row in temp_violations.iterrows():
            print(f"  {row['regra']}: {row['violacoes']:,}")

        return val_df
    else:
        print(f"❌ Arquivo não encontrado: {valid}")
        return None

def inspecionar_histogramas():
    """Verifica distribuições para identificar variáveis problemáticas"""
    print("\n" + "="*60)
    print("4. ANÁLISE DE HISTOGRAMAS")
    print("="*60)

    if os.path.exists(hist):
        hist_df = pd.read_csv(hist)

        # Verificar variáveis quase constantes
        print("Verificando variáveis potencialmente constantes:")

        vars_interesse = [
            'potencia_gerada_estimada_mw', 'wind_shear',
            'intensidade_turbulencia', 'velocidade_vento_10m',
            'velocidade_vento_100m'
        ]

        for var in vars_interesse:
            var_hist = hist_df[hist_df['coluna'] == var]
            if not var_hist.empty:
                total_count = var_hist['count'].sum()
                max_bin_count = var_hist['count'].max()
                concentration = max_bin_count / total_count if total_count > 0 else 0

                print(f"\n{var}:")
                print(f"  Total registros: {total_count:,}")
                print(f"  Concentração máxima: {concentration:.3f}")

                if concentration > 0.9:
                    print("  ⚠️  VARIÁVEL QUASE CONSTANTE!")
                    # Mostrar os bins com mais dados
                    top_bins = var_hist.nlargest(3, 'count')
                    for _, bin_row in top_bins.iterrows():
                        print(f"    Bin [{bin_row['bin_start']:.3f}, {bin_row['bin_end']:.3f}]: {bin_row['count']:,} ({bin_row['count']/total_count:.1%})")

        return hist_df
    else:
        print(f"❌ Arquivo não encontrado: {hist}")
        return None

def calcular_limites_clipping(stats_df, percentis_df=None):
    """Calcula limites de clipping baseado nos dados disponíveis"""
    print("\n" + "="*60)
    print("5. LIMITES DE CLIPPING PROPOSTOS")
    print("="*60)

    limites_clipping = {}

    # Variáveis que precisam de clipping
    clipping_vars = {
        'wind_shear': 'P1-P99',
        'intensidade_turbulencia': 'P1-P99',
        'velocidade_vento_10m': 'P1-P99',
        'velocidade_vento_100m': 'P1-P99',
        'potencia_gerada_estimada_mw': 'P99'
    }

    for var, tipo_clip in clipping_vars.items():
        print(f"\n{var} ({tipo_clip}):")

        # Tentar usar percentis calculados primeiro
        if percentis_df is not None and var in percentis_df.index:
            p1 = percentis_df.loc[var, 'p1']
            p99 = percentis_df.loc[var, 'p99']
            print(f"  Usando percentis calculados: P1={p1:.3f}, P99={p99:.3f}")

            if tipo_clip == 'P1-P99':
                limites_clipping[var] = {'min': p1, 'max': p99}
            else:  # P99 apenas
                limites_clipping[var] = {'min': None, 'max': p99}

        # Senão, usar aproximação por média ± N*std
        else:
            var_stats = stats_df[stats_df['coluna'] == var]
            if not var_stats.empty:
                row = var_stats.iloc[0]
                mean = row['mean']
                std = row['std']

                # Aproximação: P1 ≈ média - 2.33*std, P99 ≈ média + 2.33*std
                p1_approx = mean - 2.33 * std
                p99_approx = mean + 2.33 * std

                print(f"  Usando aproximação estatística:")
                print(f"    Média: {mean:.3f}, Std: {std:.3f}")
                print(f"    P1 aprox: {p1_approx:.3f}, P99 aprox: {p99_approx:.3f}")

                # Aplicar limites físicos sensatos
                if 'velocidade_vento' in var:
                    p1_approx = max(0, p1_approx)  # Vento não pode ser negativo
                    p99_approx = min(60, p99_approx)  # Vento raramente > 60 m/s
                    print(f"    Com limites físicos: P1={p1_approx:.3f}, P99={p99_approx:.3f}")

                if tipo_clip == 'P1-P99':
                    limites_clipping[var] = {'min': p1_approx, 'max': p99_approx}
                else:  # P99 apenas
                    limites_clipping[var] = {'min': None, 'max': p99_approx}
            else:
                print(f"  ❌ Estatísticas não encontradas para {var}")

    return limites_clipping

def contar_arquivos_parquet():
    """Conta quantos arquivos parquet serão processados"""
    print("\n" + "="*60)
    print("6. ARQUIVOS PARA PROCESSAMENTO")
    print("="*60)

    pattern = os.path.join(serie_temporal, "series_temporais_*.parquet")
    parquet_files = glob(pattern)

    print(f"Padrão de busca: {pattern}")
    print(f"Arquivos encontrados: {len(parquet_files)}")

    if parquet_files:
        print("Primeiros 5 arquivos:")
        for i, arquivo in enumerate(parquet_files[:5]):
            nome = os.path.basename(arquivo)
            tamanho_mb = os.path.getsize(arquivo) / (1024*1024)
            print(f"  {i+1}. {nome} ({tamanho_mb:.1f} MB)")

        if len(parquet_files) > 5:
            print(f"  ... e mais {len(parquet_files) - 5} arquivos")

    return parquet_files

# Executar todas as inspeções
print("INSPEÇÃO COMPLETA DOS RESULTADOS DA EDA")
print("="*60)

percentis = inspecionar_percentis_globais()
stats = inspecionar_estatisticas_globais()
validacoes = inspecionar_validacoes()
histogramas = inspecionar_histogramas()

if stats is not None:
    limites = calcular_limites_clipping(stats, percentis)
    print(f"\nLIMITES FINAIS DE CLIPPING:")
    for var, limits in limites.items():
        if limits['min'] is not None:
            print(f"  {var}: [{limits['min']:.3f}, {limits['max']:.3f}]")
        else:
            print(f"  {var}: [sem limite inferior, {limits['max']:.3f}]")

arquivos = contar_arquivos_parquet()

print("\n" + "="*60)
print("INSPEÇÃO CONCLUÍDA")
print("="*60)

INSPEÇÃO COMPLETA DOS RESULTADOS DA EDA
1. PERCENTIS GLOBAIS
Percentis calculados:
                Unnamed: 0        p1       p99
0               wind_shear  0.069626  0.368807
1  intensidade_turbulencia  0.018654  0.067647

Variáveis com percentis: [0, 1]
Variáveis de velocidade de vento encontradas: []
⚠️  ATENÇÃO: Percentis de velocidade de vento NÃO foram calculados!

2. ESTATÍSTICAS GLOBAIS
Estatísticas das variáveis de interesse:
--------------------------------------------------------------------------------

wind_shear:
  Count: 3,404,160
  Min/Max: 0.034 / 0.628
  Média ± Std: 0.175 ± 0.068
  P1/P99 aproximados: 0.016 / 0.334

intensidade_turbulencia:
  Count: 3,404,160
  Min/Max: 0.010 / 0.102
  Média ± Std: 0.041 ± 0.009
  P1/P99 aproximados: 0.020 / 0.062

velocidade_vento_10m:
  Count: 3,404,160
  Min/Max: 0.458 / 9.738
  Média ± Std: 5.008 ± 1.400
  P1/P99 aproximados: 1.746 / 8.271

velocidade_vento_100m:
  Count: 3,404,160
  Min/Max: 0.620 / 12.008
  Média ± Std: 7.325 